<a href="https://colab.research.google.com/github/nasirsaba7/arg-microtexts/blob/master/Copy_of_livingwithPF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the transformers package from Hugging Face:
!pip install transformers
!pip install torch
!pip install tensorflow

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

In [ ]:
# Load the model and tokenizer
model = BartForConditionalGeneration.from_pretrained(
    'facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained(
    'facebook/bart-large-cnn')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def summarize(text, maxSummarylength=500):
    # Encode the text and summarize
    inputs = tokenizer.encode("summarize: " +
                              text,
                              return_tensors="pt",
                              max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=maxSummarylength,
                                 min_length=int(maxSummarylength/5),
                                 length_penalty=10.0,
                                 num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [ ]:
def split_text_into_pieces(text,
                           max_tokens=900,
                           overlapPercent=10):
    # Tokenize the text
    tokens = tokenizer.tokenize(text)

    # Calculate the overlap in tokens
    overlap_tokens = int(max_tokens * overlapPercent / 100)

    # Split the tokens into chunks of size
    # max_tokens with overlap
    pieces = [tokens[i:i + max_tokens]
              for i in range(0, len(tokens),
                             max_tokens - overlap_tokens)]

    # Convert the token pieces back into text
    text_pieces = [tokenizer.decode(
        tokenizer.convert_tokens_to_ids(piece),
        skip_special_tokens=True) for piece in pieces]

    return text_pieces

In [ ]:
def recursive_summarize(text, max_length=200, recursionLevel=0):
    recursionLevel=recursionLevel+1
    print("######### Recursion level: ",
          recursionLevel,"\n\n######### ")
    tokens = tokenizer.tokenize(text)
    expectedCountOfChunks = len(tokens)/max_length
    max_length=int(len(tokens)/expectedCountOfChunks)+2

    # Break the text into pieces of max_length
    pieces = split_text_into_pieces(text, max_tokens=max_length)

    print("Number of pieces: ", len(pieces))
    # Summarize each piece
    summaries=[]
    k=0
    for k in range(0, len(pieces)):
        piece=pieces[k]
        print("****************************************************")
        print("Piece:",(k+1)," out of ", len(pieces), "pieces")
        print(piece, "\n")
        summary =summarize(piece, maxSummarylength=max_length/3*2)
        print("SUMNMARY: ", summary)
        summaries.append(summary)
        print("****************************************************")

    concatenated_summary = ' '.join(summaries)

    tokens = tokenizer.tokenize(concatenated_summary)

    if len(tokens) > max_length:
        # If the concatenated_summary is too long, repeat the process
        print("############# GOING RECURSIVE ##############")
        return recursive_summarize(concatenated_summary,
                                   max_length=max_length,
                                   recursionLevel=recursionLevel)
    else:
      # Concatenate the summaries and summarize again
        final_summary=concatenated_summary
        if len(pieces)>1:
            final_summary = summarize(concatenated_summary,
                                  maxSummarylength=max_length)
        return final_summary

In [ ]:
# Example usage - living with PF
text = '''I was diagnosed with Idiopathic Pulmonary Fibrosis, also known as IPF. I was only 28 years old.
IPF has been a surprise and a life-altering challenge. I’m nearly 71 years old, and was formally diagnosed three years ago.
At the age of 4, I was diagnosed with juvenile rheumatoid arthritis… we counted ourselves lucky that it appeared I had the presentation where it attacks primarily the joints. We didn’t realize this would later take our journey down another path… My childhood had its aches and pains. After college was when things really started to change. I was diagnosed with double pneumonia shortly after graduating… Blood tests, CT scans and a lung biopsy eventually confirmed a diagnosis of pulmonary fibrosis in 2011. My family and I later found out that rheumatoid arthritis caused my pulmonary fibrosis.'''

final_summary = recursive_summarize(text)
print("\n%%%%%%%%%%%%%%%%%%%%%\n")
print("Final summary:", final_summary)

######### Recursion level:  1 

######### 
Number of pieces:  1
****************************************************
Piece: 1  out of  1 pieces
I was diagnosed with Idiopathic Pulmonary Fibrosis, also known as IPF. I was only 28 years old.
IPF has been a surprise and a life-altering challenge. I’m nearly 71 years old, and was formally diagnosed three years ago.
At the age of 4, I was diagnosed with juvenile rheumatoid arthritis… we counted ourselves lucky that it appeared I had the presentation where it attacks primarily the joints. We didn’t realize this would later take our journey down another path… My childhood had its aches and pains. After college was when things really started to change. I was diagnosed with double pneumonia shortly after graduating… Blood tests, CT scans and a lung biopsy eventually confirmed a diagnosis of pulmonary fibrosis in 2011. My family and I later found out that rheumatoid arthritis caused my pulmonary fibrosis. 

SUMNMARY:  IPF has been a surprise and

In [ ]:
# Example usage - Symptoms, comorbidities and physical/cognitive limitations
text = '''Breathlessness is really difficult to describe. I can only liken it to feeling like you’re under water and want to breathe, but you can’t. I’ve had to learn techniques to help me manage my breathlessness.
[Coughing] can cause me to vomit ... that impacts on myself and my family, keeping us awake at night particularly.
Feelings of weakness and breathlessness overtake the enjoyment and ability to do much.
I found myself persistently short of breath, fatigued, and dealing with a cough that wouldn’t subside.
Coughing all the time is exhausting. I have little energy to do as I like, like spending time with my grandchildren.
Patients also report weight loss as a symptom of PF:
Weight loss seems to be part and parcel of IPF and mine has dropped from 90kg to about 67kg now, so you have to eat what you need to maintain health.
That’s been progressive and weight loss ... and very low BMI and advanced nutritional support.
If it’s really cold outside… it hurts me to go outside. It really hurts my lungs. Heat and humidity really, really take their toll. Allergy season is crazy.
Patients’ also describe the progressive worsening of symptoms over time:
I first noticed my breathing problems on a long-haul flight and struggled with my breathing while I was away. I thought it was a virus, but after some planned surgery the following month I developed a cough, struggled with my breathing and was admitted to the emergency department ... At times I’d feel fine, then going upstairs would leave me incredibly breathless. I developed a persistent cough and felt tired a lot of the time.
As the disease progressed the coughing became more prevalent. It affected simple day acts; I’d cough when I laughed, after eating and in atmospheres where there are smells of chemicals, perfumes and aerosol deodorants, or spaces too hot or cold. The cough has increased and sounds different as the time passes. The sound of my cough is more grave and I now cough for longer periods.
Flare-up of symptoms is reported to be severely debilitating:
In May 2018 I had a flare-up of my symptoms while in Rome due to a respiratory infection. I was coughing all day and night, and couldn’t sleep. It took about 8 weeks and some pretty strong antibiotics to shift it completely.
I have a constant dry cough. Very annoying. I am on supplemental oxygen 24 hours a day. I even have canisters that I use when I go outside of the house. But it’s a pain... I have some days where it’s not really that bad. I have some days where it is so severe that it’s – I cough to the point that I throw up because I’m coughing so much.
My symptoms were heavy labored breathing when I did normal tasks such as climbing a flight of stairs and numbness and discoloration in my toes and fingers or Raynaud’s syndrome.
I feel like the last couple of months I’ve been feeling more fatigue than I have in the past… Sometimes there [are] things I have to do, so I kind of work through it, but, you know, when I, when I don’t lie down and I still go to work and still do things, it drains me even more.
Sometimes I get chest pain. It comes and goes, but it mostly feels like somebody really heavy, like I feel a 400-pound person, like I got an extra body sitting on me.
I couldn’t walk along freely without oxygen. I had to have somebody nearby just in case, because I have these dizzy spells and sometimes I would pass out.
It’s just one of those feelings where [it’s] kind of like you panic. You know, you want air in. Because you need air to breathe, and if you don’t have air you just feel all discombobulated. That’s kind of a weird feeling.
I was diagnosed in 2007 with Pulmonary Fibrosis because of Lupus an autoimmune disease which caused the damage to my lungs.
Patient with PF caused by antibiotics for urinary tract infection:
I got the Fibrosis, which is life shortening, by repeatedly being prescribed an antibiotic for urinary tract infection. The antibiotic is Nitrofurantoin.
Patients with PF caused by taking medications for their atrial fibrillation:
I have pulmonary fibrosis caused by the treatment I had for AF. I had amiodarone, which is known to cause lung problems, and then dronedarone, which is safer. Unfortunately, I developed pulmonary fibrosis.
I too have pulmonary fibrosis, diagnosed 4 years ago and caused by the drug I was taking for my atrial fibrillation, which was then stopped, but the damage was done. I also have other health problems with my heart and lungs.
Subpopulation of special interest SPOTLIGHT on SSc-ILD
Comorbidities reported by patients
In a study of 231 patients with SSc-ILD, patients reported comorbidities including Raynaud’s syndrome, GI tract issues, skin symptoms, dysphagia disorder, cardiovascular issues (tachycardia, hypertension, pulmonary, chest pain), anaemia, fibromyalgia, arthritis and cognitive issues.
Specific comments from patients with SSc-ILD include:
When I initially started, I was itching every day, and it would keep me up at night. It was just constantly itching.
At the beginning, when I was first diagnosed, I used to be a really, severe Raynaud’s where just touching the fingers were painful.
I also have severe osteoarthritis in both knees.
Where now, I don’t have that sensitivity in the fingertips. It will turn purple; it will hurt but not as bad as in the beginning when I was diagnosed. I used to get sores-I used to get digital ulcers and scars and sores, and now I don’t. Fingers turn blue but I don’t get the sores.
I have Raynaud’s, my fingers are curling up and I have lost my nails, pulmonary fibrosis along with the scleroderma, acid reflux, aspergillosis and an irregular heart.'''

final_summary = recursive_summarize(text)
print("\n%%%%%%%%%%%%%%%%%%%%%\n")
print("Final summary:", final_summary)

######### Recursion level:  1 

######### 
Number of pieces:  8
****************************************************
Piece: 1  out of  8 pieces
Breathlessness is really difficult to describe. I can only liken it to feeling like you’re under water and want to breathe, but you can’t. I’ve had to learn techniques to help me manage my breathlessness.
[Coughing] can cause me to vomit... that impacts on myself and my family, keeping us awake at night particularly.
Feelings of weakness and breathlessness overtake the enjoyment and ability to do much.
I found myself persistently short of breath, fatigued, and dealing with a cough that wouldn’t subside.
Coughing all the time is exhausting. I have little energy to do as I like, like spending time with my grandchildren.
Patients also report weight loss as a symptom of PF:
Weight loss seems to be part and parcel of IPF and mine has dropped from 90kg to about 67kg now, so you have to eat what you need to maintain health.
That� 

SUMNMARY:  Patients

In [ ]:
# Example usage - Diagnosis, referral and treatment pathways
text = '''I started experiencing a buildup of mucus in my throat causing a constant clearing and coughing. The colour was always clear so, since both my family doctor and I felt it was post nasal drip, I was prescribed nasal sprays.
Before my diagnosis I’d been feeling some pain on my lower back that wouldn’t go away. I also noticed that I often needed to cough to clear my throat before having a conversation but I wouldn’t call it severe.
It took around 2 weeks for her to be diagnosed after being taken to A&E at Hammersmith Hospital, London when she felt her breathing was not right. I remember being told at the time that her lungs were too weak for a biopsy to be done to confirm her diagnosis. Shortly after, she was given a prognosis of 5 years. This was a massive shock and we felt totally winded. It was unexplainable for us and felt surreal.
I first mentioned my cough to my GP at my annual health check in 2013. The X-ray came back clear. But a year later, another X-ray showed something on my left lung. Lots of tests were inconclusive, but finally the results of a surgical lung biopsy indicated I had IPF.
It took 2 years of back and forth to get to this stage!
Then we moved and I saw a new GP. I was sent for a CT scan almost immediately, which showed that there was damage in my airways. I was told I had pulmonary fibrosis and referred to hospital. After a lung biopsy and further tests I was finally diagnosed, three years after my first visit to the GP.
At the age of 4, I was diagnosed with juvenile rheumatoid arthritis… we counted ourselves lucky that it appeared I had the presentation where it attacks primarily the joints. We didn’t realize this would later take our journey down another path… My childhood had its aches and pains. After college was when things really started to change. I was diagnosed with double pneumonia shortly after graduating… Blood tests, CT scans and a lung biopsy eventually confirmed a diagnosis of pulmonary fibrosis in 2011. My family and I later found out that rheumatoid arthritis caused my pulmonary fibrosis.
I was diagnosed with double pneumonia shortly after graduating. After recovering, I still had a cough that I couldn’t shake no matter how many over-the-counter or prescription edications I tried, and while I was far from an athlete before, the shortness of breath was slightly more noticeable now.
At first I was told I had arthritis, then rheumatoid arthritis. Lupus was also diagnosed and I was given treatments for them all. Nothing helped and I only got worse ... he ran a myriad of tests; X- rays, blood and urine ... and told us that I had scleroderma with CREST and pulmonary fibrosis.
My breathing complications became really bad so my chest specialist thought that I had a condition called bronchiectasis and did [an] MRI scan to detect it. This came back clear of bronchiectasis but he said we did find pulmonary fibrosis on the scan.
My dad was misdiagnosed for five years and was finally told he had IPF one week before he died.
I changed respirologists part way through (the process of being treated) because I wasn’t getting the information I wanted.
Doctor (diagnoses) with fibermyalgia [sic] and prescribed prednisone 40mg… My arthritis results came back negative… Upon visiting a physician he noticed that my oxygen is very very low… had CT scan with contrast of my entire chest where they discovered fibrosis.
I was first diagnosed with bronchiectasis in 2002. It took till 2011 before I discovered that I had pulmonary fibrosis.
I got a really bad respiratory flu. I didn’t recover in 10 or 12 days… and at first they thought it was COPD.
My health had been deteriorating for quite a while running up to my IPF diagnosis in 2015. In early 2014, I had been experiencing severe chest pains. An x-ray revealed honeycombing on my lungs and my doctor could hear crackling. Following a lung biopsy at hospital, my partner and I were initially told that I did not have IPF – we left the hospital feeling relieved. Unfortunately, the relief was short-lived. At my next routine appointment, another doctor informed me that I did have IPF. This came as a massive shock and I left the hospital feeling unhappy and anxious about the future. When I got home, I told Allison, who was distraught; having previously read that the average life expectancy from diagnosis was 3-5 years.
Very shocked. Really upset at the time. Coming to terms with it slowly.
They gave me a booklet and I wish I didn’t read it.
To give it to someone as soon as they diagnosed it, I think it was not a good idea.
While the diagnosis itself might be clearly explained at the time, patients report a subsequent lack of support and information:
After diagnosis, I have been left on my own to cope, with no help from anyone.
Things were very confusing at this time for me, and I remember wondering why I was being admitted, because I didn’t feel particularly worse than I had in the months prior.
I felt desperate and alone. After such an awful diagnosis, my next appointment was in 6 months and nothing in between.
To be diagnosed with IPF is absolutely devastating ... You are told you have three to five years to live ... It’s a very, very scary proposition.
I guess the reason why I felt so surprised was that there was no mention of pulmonary fibrosis in the years prior, when I had multiple episodes of flu, bronchitis, pneumonia.
It was a bit of a shock to the system ... we thought it was just a chest infection ... he had a little cough but nothing too much.
Initially it was a huge shock to discover that he had IPF, which I had never heard of.
Given that I am a life-long non-smoker and never knowingly worked in a hazardous environment, I couldn’t fathom my diagnosis. Why me and IPF?
I feel picked on, I feel I haven’t smoked for all these years, I don’t drink, I’m not overly fat, why is this happening to me?
My general practitioner had no idea what was wrong with me. I used to feel upset when I left his office as I knew he did not believe what I was telling him.
I was diagnosed with Interstitial Lung Disease (ILD) caused by scleroderma at the end of October in 2012… My first reaction was disbelief especially when I was told that one of the ‘treatments’ would be chemotherapy.
Blood tests, CT scans and a lung biopsy eventually confirmed a diagnosis of pulmonary fibrosis in 2011. My family and I later found out that rheumatoid arthritis caused my pulmonary fibrosis. I was shocked, but I was determined to see the bright side of things. We had a diagnosis and now could start on a path of monitoring this to hopefully prevent any further complications.
At the time of diagnosis I had no symptoms, I was athletic and completely healthy. I had no idea the impact it would come to have on my life.
I was told I had Interstitial Lung Disease and Pneumonitis which resulted in scarring and a partially collapsed left lung in early 2019 when I had been on MTX. Is that pulmonary fibrosis? I don’t know. I’m about to be referred to a Respiratory consultant (again) as I have a blood clot in each lung and so will ask for an explanation of all the different terms.
… but had always been diagnosed as something else, such as pneumonia. This unfortunately led to countless, useless prescriptions of antibiotics; to post-nasal drip; to possibly tuberculosis (simple tests proved otherwise). Luckily, my new and astute physician decided that the problem was greater and a referral to a respirologist confirmed IPF.
I was referred to a respirologist and put on a high dose steroid, and a bronchodilator inhaler. Several tests followed and finally in December 2010, based on a lung biopsy, I was diagnosed with IPF that had already reduced my lung function by 40%.
Have [sic] been diagnosed this by my respiratory consultant due to decrease in my lung function tests.
I was diagnosed with pulmonary fibrosis approx 5 years ago. My respiratory consultant believes it RA related and I’ve no reason to doubt him as I’m as sure as I can be that my lungs were damaged during a prolonged RA flare.
Am going to a connective tissue consultant… as my rheumatologist consultant thinks this would help me better.
My hospital has a scleroderma specialist, thankfully! I had been treated for pulmonary fibrosis there. Some other issues landed me with rheumatology, but they seem to have a few multidisciplinary specialists that see scleroderma patients, which is great.'''

final_summary = recursive_summarize(text)
print("\n%%%%%%%%%%%%%%%%%%%%%\n")
print("Final summary:", final_summary)

######### Recursion level:  1 

######### 
Number of pieces:  11
****************************************************
Piece: 1  out of  11 pieces
I started experiencing a buildup of mucus in my throat causing a constant clearing and coughing. The colour was always clear so, since both my family doctor and I felt it was post nasal drip, I was prescribed nasal sprays.
Before my diagnosis I’d been feeling some pain on my lower back that wouldn’t go away. I also noticed that I often needed to cough to clear my throat before having a conversation but I wouldn’t call it severe.
It took around 2 weeks for her to be diagnosed after being taken to A&E at Hammersmith Hospital, London when she felt her breathing was not right. I remember being told at the time that her lungs were too weak for a biopsy to be done to confirm her diagnosis. Shortly after, she was given a prognosis of 5 years. This was a massive shock and we felt totally winded. It was unexplainable for us and felt surreal.
I first m

In [ ]:
# Example usage - Diagnosis, referral and treatment pathways
text = '''I was diagnosed with pulmonary fibrosis in 2004 and subsequently got Pulmonary Hypertension as well. A year ago I was prescribed Hydroxychloroquine Sulfate.
I’m on a drug called nintedanib which I’ve been on for 4 years. I had to have the dose lowered to reduce side effects. We’re both also on long-term oxygen therapy.
I’ve been on nintedanib for nearly a year now. The drug isn’t expected to improve things as scarring can’t be reversed but it is hoped it will stop or slow down progression. I did feel better straight away but I am also taking steroids.
Ambulatory oxygen is very important – it helps me keep active.
I’ve recently been given an Inogen one oxygen concentrator. It only gives oxygen when you breathe through the tubes up your nose.
Patients describe having a strong support system in place as being important:
Eventually you become dependent on supplementary oxygen for exercise initially and then 24/7 at home just to stay alive.
You’re using your own supports whether it’s your friends, my telephones been burnt out. It’s just staying in touch with people, that’s kind of what gets you through the day.
Yoga, meditation, just going out for a walk, I mean all of those things definitely helped.
I started with the garden benches [home improvement project]. It’s just something to keep you going... keep the mind busy on something else… The more you sit around watching the television, the more things will get into your mind.
Patient groups also provide a powerful coping mechanism:
There was company, people with the same disease. I found I was looking forward to going. And I thought it was doing me good.
Just having a chat about families and how they are coping, you know that’s the biggest part of it. Getting together and talking to people with the same problem.
However, the influence was not consistently positive:
‘I couldn’t help thinking that [the patient group meeting] is like
[Dr Harold Shipman’s] waiting room, waiting for each one to pop off you know. Because there is always news that somebody is gone and that’s a person you know and I think, my name would be there one day wouldn’t it?’ (Dr Harold Shipman was a UK physician convicted of euthanasia of elderly patients)
 ... [There was a negative] down turn because you all know that you are going sooner than you’d like to be.
It’s sort of monitoring yourself and knowing what your capacity is, and if it’s… you’re getting to a… sort of a point where you need to take a break, well take a break you know, that’s all. I guess you learn yourself where those points are.
I think by, referring how it affects me to somebody else, may not help them at all, yet it helps me. And that’s why I’m saying, I think it’s sort of very individual.
That means that I have to be aware, more aware… and try and make sure [I] eat properly, exercise, monitor my oxygen requirements.
I have to, just get into a routine and acceptance of that, and find the little pockets of positivity within that well.
Preparing oneself for… acceptance of the illness, not just acceptance of the illness, but acceptance of the illness that where it’s going to take you, as in, in the end, you will die. But um, you can’t just put that out of your mind.
I know they (family) care and I know they’re gonna be there for me and that’s a big, a big plus, whether it’s on an emotional level or a physical level, they would do anything to help me that they could.
I can now restore my breathing by remaining as calm as possible and inhaling through my nose. […] I find that drinking water and keeping hydrated, having a small, low sugar sweet in my mouth (liquorice is quite effective) and having small meals, help me to reduce my cough.
I do find that my mouth dries out so I must do some mouth breathing, cause I’m sort of constantly putting on, you know, lip balm during the day, and at nighttime too.
My mother died of lung disease, she had fibrosis, so she had scarring all over her lungs and eventually it covered her lung and I mean, there’s no way you can breathe after a while, and I know this is mind over matter, but do you know those little propeller handheld fans? I mean, this is just mind over matter for sure, but that feeling of air just on her face calmed her down.
The decision to start treatment was mine, but it was based on my lung specialist’s advice because I am a layman.
I made the decision. I had to make my physician aware of the treatments to get them.
I asked the ILD specialist about which drug he would recommend, I was recommended Esbriet. However, I did NOT want to take a treatment that required me taking 9 tablets a day on top of all my heart medication that I already take.
My pulmonary fibrosis is due to Rheumatoid Arthritis [RA], the RA is being actively treated but constantly getting pneumonia requiring hospital admissions.
A low dose of Baracitinib for my RA. Apparently some rheumatologists don’t like prescribing baricitinib to people with lung conditions but I’ve been taking it for approx. 2 years without any detrimental effect on my lungs so far. My respiratory consultant advised me to stop taking methotrexate which I was on when diagnosed. Rheumy have stopped me taking all other DMARDs too, which I think is because of my pulmonary fibrosis.
The lack of multidisciplinary team plays a role here. For instance, physiotherapy is not contemplated as part of the process. The multidisciplinary team should not only include lung specialists, radiologists and pathologists, it should also include physiotherapists.
For a few months, I’ve been using ambulatory oxygen. I had struggled with the concept, as I saw it as a sign my condition was progressing. But now I see it as a way of allowing me to continue to do the things I want to do and as an important safeguard.
[Being on oxygen] enables you to carry on doing the things that you want to do… at least it enables you to go out and do some exercise… go off to the park, go and see friends… I think the way people think of oxygen needs to change… that it’s a positive thing.
I didn’t want to go on oxygen because it was accepting or confirming that I was getting worse. When I look back on it, I should have been on supplementary oxygen 6 months or more before.
To slow down the scarring of my lungs, I was prescribed pirfenidone. Initially I had a lot of side effects: nausea, vomiting, diarrhoea and a skin rash which meant I had to put on SPF50 sunscreen whenever I went out in the sun. I also felt very fatigued sometimes and would need to take short breaks during the day. But I tolerated all this as it was slowing the rate of progression of the scarring on my lungs.
Corticosteroids made me feel like a balloon.
Going out is always a challenge because I worry about running out of oxygen.
The three most common challenges I’ve faced are travelling with oxygen, getting the appropriate oxygen for home use, and exercising with oxygen.
I was amazed at how useful it was; it told me things I feel my GP should have; things I needed to know. It gave me the confidence to go to the gym, even with my oxygen. I met a friend on the course and we went together and both exercised with our oxygen.
At the course I found out breathing techniques that I still use today, years later. It helped me to cope and accept my condition.
I did pulmonary rehab and it really helped. But because I’ve already done it, and there’s such a waiting list, I haven’t gone back.
The doctor suggested I go again recently, and I said to the doctor I don’t really feel I can do it. Or have the energy. And then my husband would have to drive me, and it’s just sheer distance.
What really helped me was going to a pulmonary rehab program. That was a way of being able to start getting back into exercising, and activity, and wellness exercises, and connecting with other people who have lung issues.
The fitter your body is, the better you feel both physically and mentally. If you feel better mentally then your whole well-being is more positive and you won’t dwell on the negative sides of the disease.
Palliative care can be very useful… you can get medication… assistance, help with nutritional changes, and also participate in relaxation techniques, and get advice on social, emotional and spiritual… Psychological support is very, very important.
I often forget to take the one pill I have to take weekly.
I prefer to experience diarrhoea than a headache as my line of work requires lots of concentration. I tend to gain weight — specifically, corticosteroids made me feel like a balloon. I believe that at the end of the day… any side effects worsen quality of life.
If they are all severe, diarrhoea is the hardest… every moment of the day going to the toilet can be embarrassing and hits that physical and pride level.
The most common thing that a lot of people can’t manage is the diarrhoea, because you’re literally at home and you can’t go anywhere.
The sun sensitivity is especially inconvenient as my wife comes from the Caribbean and still wants to travel there and then it would be severe to travel there with such a risk of side effects.
I’m already on a cocktail of pills and taking nine more a day is too much to ask.
However, in June 2017, test results indicated that the medication wasn’t slowing the scarring as much as was hoped. So, I was switched to the only other available and relatively new drug, nintedanib. I was a bit nauseous initially, but I’ve found a good regime and manage my diet.
I was prescribed Esbriet at first but experienced intolerable side effects so the drug was exchanged for OFEV. Is it working? Well, my spirometry test shows little minimal change in the last 6 months.
Some people like me don’t can’t seem to absorb or respond to the Zentevia brand and only do well with the Bristol Brand/Quinoric Brand… Mine was switched in July and I landed up in hospital for 6 weeks. I never dreamt that it could be due to a medication brand switch but it was.
There are only 2 approved medications to help slow the condition and I have been on both – the first, Pirfenidone, was not as effective as hoped and so I was switched to Nintedanib.
Stephen preferred to have diarrhoea and only 2 pills a day instead of vomiting and 9 pills.
Oh, over the phone is perfect for me because I don’t want to be going in there at all… the hospital is the worst place in the world to be in these days.
There was great support in the knowledge that if I was unwell I could ring… That message was re-enforced many times, that if I needed a bit of help, someone to talk to.
Negative
You really do miss those (faceto-face) meetings… when you come away from it, it’s absolutely brilliant, it’s just the confidence.
I’d be praying night and day that there would not be any changes in my condition… I’d be often worried… I would not be able to self-diagnose myself.
For example, if you go to physio, they tell you how to do things [exercises] and then by the time you get home you’ve forgotten it. Whereas, if you’ve got a video, you can always fall back on it and make sure you are doing it correctly.
When you go to the doctors you get like seven minutes, the seven minutes finish and I have forgotten, you know what I mean. If you had something on the internet, you could watch it over and over an explain it, you know what I mean, so you can understand it.
I’ve often wondered to what extent the diagnosis is exactly correct. When I go online and read about my symptoms, I often find that the same symptoms apply to several diseases under the same umbrella. I’m diagnosed with IPF… some of the treatments seem to be a little bit different. It would be probably quite helpful if there were a genetic test that would say, ‘yeah, this is what you have, not that. And so your treatment should be (this).
It would be a real advantage if there was genetic testing developed that might catch something like pulmonary fibrosis right at the very beginning and start treatment right away instead of having it develop and cause lots of scarring before it’s diagnosed.
I think it would depend on how invasive it is and how accurate it is. I wouldn’t want it to leave me with more questions that there are no answers for.
I have siblings, one issue I think is whether … somebody would actually want to know… Some people would want to live their life accordingly. Others, if they think they’re going to get this terrible disease, they’ll be devastated, and it would really negatively affect their quality of life.
The sun sensitivity is especially inconvenient as my wife comes from the Caribbean and still wants to travel there and then it would be severe to travel there with such a risk of side effects.
However, in June 2017, test results indicated that the medication wasn’t slowing the scarring as much as was hoped. So I was switched to the only other available and relatively new drug, nintedanib. I was a bit nauseous initially, but I’ve found a good regime and manage my diet.
I was prescribed Esbriet at first but experienced intolerable side effects so the drug was exchanged for OFEV. Is it working? Well, my spirometry test shows little minimal change in the last 6 months.
Some people like me don’t can’t [sic] seem to absorb or respond to the Zentevia brand and only do well with the Bristol Brand/Quinoric Brand … Mine [medication] was switched in July and I landed up in hospital for 6 weeks. I never dreamt that it could be due to a medication brand switch but it was.
There are only 2 approved medications to help slow the condition and I have been on both – the first, Pirfenidone, was not as effective as hoped and so I was switched to Nintedanib.
Stephen [the patient] preferred to have diarrhoea and only 2 pills a day instead of vomiting and 9 pills.'''

final_summary = recursive_summarize(text)
print("\n%%%%%%%%%%%%%%%%%%%%%\n")
print("Final summary:", final_summary)

######### Recursion level:  1 

######### 
Number of pieces:  19
****************************************************
Piece: 1  out of  19 pieces
I was diagnosed with pulmonary fibrosis in 2004 and subsequently got Pulmonary Hypertension as well. A year ago I was prescribed Hydroxychloroquine Sulfate.
I’m on a drug called nintedanib which I’ve been on for 4 years. I had to have the dose lowered to reduce side effects. We’re both also on long-term oxygen therapy.
I’ve been on nintedanib for nearly a year now. The drug isn’t expected to improve things as scarring can’t be reversed but it is hoped it will stop or slow down progression. I did feel better straight away but I am also taking steroids.
Ambulatory oxygen is very important – it helps me keep active.
I’ve recently been given an Inogen one oxygen concentrator. It only gives oxygen when you breathe through the tubes up your nose.
Patients describe having a strong support system in 

SUMNMARY:  I’m on a drug called nintedanib which 

In [ ]:
# Example usage - Specific impacts of PF on activities of daily living including navigating healthcare environment and access
text = '''As IPF was destroying my dad’s life, he struggled with the disappointment that no cure existed, and he looked for a way to support research that could lead to a cure.
‘I just wish it would stop. The feeling that it is never going to improve is sometimes overwhelming.
I used to play a lot of golf, swimming, cricket, you name it. I’ve had no problem, but now I’m a television viewer.
I’ve sometimes found lifting whatever across the garden these days is difficult, but before it was easy.
I try to keep everything in precise order, so things at the same place all the time.
It’s sort of reorganizing your life, try and do everything in one place before you move to another.
There are occasions when I can’t participate in events because I’m too fatigued and I’ve learned that it’s important to pay attention to my body when it’s telling me to rest ... I am determined not to let this disease steal my life… … The pain and distressing fatigue made it difficult to exercise, as well, which led to decreased function and more weight gain. I felt trapped in a distressing cycle of illness ... The damage to my lungs from the sarcoidosis has left me with many limitations but I try not to allow these limitations to stop me from living my best life. I work full- time, I travel, I exercise and interact with my friends and family.
I am on oxygen all the time, and can only walk a short distance. Because my left lung collapsed due to so much scar tissue, I have partially lost my voice.
I limit my physical activities, take measures to keep my hands and feet warm at all times, avoid unnecessary physical contact with people, and avoid eating out. I also try to limit my travel.
When I was first diagnosed [with IPF], we made significant lifestyle changes, we downsized our house in preparation and immediately booked all the holidays we could. I now only work part time, which enables me to work around the many hospital appointments. Allison accompanies me on most appointments for moral support and to listen to the detail.
Under occupational health advice, Charlie’s employer has altered his role so he is office based now and has reduced working hours.
I find it very frustrating that I cannot do the amount of manual tasks I used to do and quite often have to leave jobs unfinished for another time. Jobs I would previously do without thinking now have to be planned and timed. Some tasks I just can’t do at all anymore which goes against a lifetime of independence.
Daily routines like taking a shower, brushing my teeth, or getting dressed are a full time job.
As the disease progressed, it became more exhausting. I couldn’t do anything without oxygen. Everything I try to do, which would usually take a day to do, takes me several days instead.
I started to notice the effects of the disease and how difficult very simple tasks such as brushing my teeth, taking a shower, putting on clothes, chronic coughing with clear mucus, walking, and climbing stairs or inclines had become.
I get breathless doing everyday things like making a cup of tea. It can be quite isolating, but you’ve got to keep going and learn how to adapt to your new norm.
Sheetal became breathless when on a long walk or climbing stairs, but she was able to continue working, do the shopping, daily chores and look after Naisha. She wanted to live and be there for Naisha and was able to live a fairly normal life with her symptoms. She was a very strong and courageous person, in fact, the bravest woman I’ve known. Nothing phased her. But it must have been incredibly difficult for her living in the knowledge that she had limited time– particularly as a mother who wanted to be there to see her daughter grow up.
You effectively become disabled in your own house. You need care. You need to have somebody to help you with simple tasks like washing and dressing.
My breathing became laboured and simply walking exhausted me — stairs, hills and steep grades were killers. Basic chores involving any form of exertion not only slowed my ability to do things but my desire to do them.
I am on oxygen 24/7 but I still struggle with day-to-day activities.
Initially, you notice it because you find, you know, walking up hills difficult… then over time even walking up the stairs ... slowly it becomes difficult even to walk on the flat.
My activities are severely limited now. Going and taking in my recycling bin requires supplementary oxygen.
I can’t get out to get the paper every day, so I read the paper every morning, virtually.
I took time off from work and worked from home on many occasions.
I began to feel isolated and ostracized because of the cough. I was embarrassed by the shortness of breath. I was irritable and suffered from insomnia because of the Prednisone that I was on to control my symptoms, and I was depressed about the weight it caused me to gain.
Well, if you can’t breathe, you can’t do anything. And at this point of the journey, that’s about where I am. I can do no housework, I can’t even wash my hair. I have to plan everything that I’m going to do that’s going to take energy.
I had scarring after severe ARDS – it’s nickname is concrete lung – the lung becomes so rigid. The medical staff presumed I would lose a great deal of lung capacity. After some brilliant hospital physiotherapy, a year after ICU, my lungs were fairly normal. Unfortunately a pleural effusion of my left lung through having my leukaemia colonise this area, has left me slightly breathless but that was 2yrs later.
I am on hospice care now and have found it to be so helpful and encouraging. I am 77, but mostly enjoy life.
We adapted to a slower pace of life. Changes were made to our home to make it easier for us both to move around. We got used to a different social life, as I was unable to leave Peter for more than 2 hours at a time.
I try not to do things that trigger my cough, which is everything. I’m embarrassed with other people’s reactions as they don’t understand my condition.
[In the context of the COVID-19 pandemic] all that year, I had been pulling away from social interaction… So when March 2020 came along it was like more of the same.
… I’m trying to fill my time and concentrate on something else worthwhile, the rugby club and the APF charity fit the bill!
We’ve also become involved with the APF support group which is great for connecting with and helping others.
 Because who wants to go out, first thing they think is that I’m contagious, and I’m coughing all over the place [… ] the first thing people think is why can’t she do something about that cough? ... But they don’t understand what the problem is, why I can’t stop coughing, and it does, you know, I’m not going to say I get embarrassed by it, but I don’t want anybody to feel [un]comfortable [… ] You know, like recently there was something, my granddaughter received an award, and I couldn’t go. So I just made the choice not to go. But it has impacted my life. I man, I’m pretty much home bound.
Sue became ill in her early fifties with pulmonary fibrosis. Very quickly life changed for us. We’ve been together since we were teenagers. We were devastated when we received the diagnosis of IPF. I’m always reminding myself that Sue used to be able to do everything and now she can’t I have to be an extension of her. She still has the same strong opinions, emotions and aspirations. I’ve learnt to be more patient, tolerant and understanding.
Caring simply means you just help them achieve whatever it is they want to do. Being a carer for your partner pulls on the heart strings and tests all the bonds you have relied on throughout your life together. I feel empowered by understanding the disease and treatments.
For me and my wife the two most difficult parts of living with scleroderma are 1) the sense of uncertainty and 2) the coordination of care among the many doctors and facilities involved. The sense of uncertainty makes us fearful of the real and imaginary symptoms that we see in me.
‘I got told off by my daughter for cutting the grass. But I got to do it. I got to do it when I get the chance.’
‘My daughter knows, my husband knows. He has been very protective in a loving way. My son, haven’t told him.’
‘My son had a shock when he saw me with the oxygen.’
‘Eventually she is talking about us moving downstairs and staying downstairs once it gets to a certain point—well I don’t want that. I’d rather take 10 minutes to go upstairs to go to bed than staying downstairs all the time.’
‘What consumes my husband controls my life as well.’
‘It soon became about logistics and practicalities and how our home would need to be adapted with a bedroom downstairs and wider door frames and ramps for wheel chair access.’
‘At the time, we didn’t feel the full force of what this meant. It was scary to think that my dad had only possibly 5 years to live, but because he hadn’t changed much (apart from a consistent cough), the terminal diagnosis hadn’t quite hit home ... His use of oxygen tanks was a big adjustment at first, and at times was difficult to see. But we knew he needed it to continue being as active as possible.’
‘Pulmonary fibrosis has hit my family hard, but the magnitude and impact on families grows exponentially when you multiply that by as many as 500,000 deaths every decade in the US alone.’
‘I’ve had some truly frightening experiences where coughing has caused my throat to close and, for a few seconds, I can’t inhale or exhale. This lasts only a few seconds but it is terrifying for my family.’
‘It affects your whole family ... our children, our extended friends, everybody’s lives were tinged by the illness that I was going through.’
‘I have tried to talk about this (end of life care) with my wife, but she interrupted me suddenly; she does not even want to hear me talk.’
In Spain it is necessary to improve early diagnosis of IPF. It is also necessary to create specific Multidisciplinary Centers that would allow an adequate follow-up and treatment to those suffering from the disease.
As there is no national transplant waiting list, it is not guaranteed that the person who needs the transplant most will get an available organ first.
One of his biggest fears is that he will miss the ‘window of opportunity’ to go on the waiting list. If his health declines too much, he might no longer be suitable: he needs to be sick enough to need a transplant, but not too sick to prevent the transplant going ahead.
Most oxygen providers will provide a home concentrator and either tanks or a portable oxygen concentrator (POC). Initially I ended up purchasing a POC out of pocket because it was difficult to get through insurance. Tanks are very heavy and, depending on your flow rate, you may have to carry more than one tank with you. Last fall I was prescribed a POC with a low continuous flow capability and it took over two months, and several follow-up calls by me… oxygen deliveries can be unreliable and sometimes require that you be home for a full day.
I wanna know how I’m doing you know, I want to be able to see this is what it was last time I did the test and this is what it is now.
I’ve got every reading I’ve ever done for the last 3 or 4 years. So, when my boy graphed it on the computer so we can see exactly where I’m at, how I’m going, you know whether my gas transfer is going up or down, do that type of thing. And within say right, maybe I gotta do a bit more exercise or whatever else, it’s a terrific gauge.
I sort of keep my phone, does the counting actually, number of steps and kilometres walked so, that’s a rough guide to how much I’ve done.
Sometimes the caregiver really knows … living with someone, I think you have a better perspective. And I think it’d be valuable for you to get the caregiver’s opinion … because they’re with them all the time.'''

final_summary = recursive_summarize(text)
print("\n%%%%%%%%%%%%%%%%%%%%%\n")
print("Final summary:", final_summary)

######### Recursion level:  1 

######### 
Number of pieces:  16
****************************************************
Piece: 1  out of  16 pieces
As IPF was destroying my dad’s life, he struggled with the disappointment that no cure existed, and he looked for a way to support research that could lead to a cure.
‘I just wish it would stop. The feeling that it is never going to improve is sometimes overwhelming.
I used to play a lot of golf, swimming, cricket, you name it. I’ve had no problem, but now I’m a television viewer.
I’ve sometimes found lifting whatever across the garden these days is difficult, but before it was easy.
I try to keep everything in precise order, so things at the same place all the time.
It’s sort of reorganizing your life, try and do everything in one place before you move to another.
There are occasions when I can’t participate in events because I’m too fatigued and I’ve learned that it’s important to pay attention to 

SUMNMARY:  I used to play a lot of golf, 

In [ ]:
# Example usage - Patient experiences and needs related to evidence/translation
text = '''I hope my participation in the [Pulmonary Fibrosis Foundation] Registry can ultimately help other patients understand and plan for what their disease may bring. How do you learn to live a fairly productive life when you have a progressive disease and there’s no playbook on what to expect? By filling out quality-of-life questionnaires for the Registry, I want to provide enough data points so that healthcare providers can help their patients work through their struggles.
Nevertheless, questions remained, and one of them was worrisome enough to get people to consider an unconventional approach to follow-up research. The problem is that the results that come out of drug studies in mice almost never translate to humans… As a result, we have close to thirty failed drug trials in pulmonary fibrosis alone.
I take part in every research study I can… Maybe that information won’t help me, but it may help someone else, maybe my children or grandchildren.
Now for the pulmonologist, the breathing is that big target because it’s a measurable thing, but for me the improvement, the quality of life is, can I walk upstairs better, can I perform things better, can I take a shower and put on clothes at the same time within the same hour? Those are things that are important to me.
I struggle with the pulmonary function tests just as my illness progresses the pulmonary function tests become more and more challenging … It’d be great if … somebody could ... come up with a better … a different PFT
[In relation to caregiver assessments and caregiver reported outcomes] Sometimes the caregiver really knows … living with someone, I think you have a better perspective. And I think it’d be valuable for you to get the caregiver’s opinion … because they’re with them all the time.
But before you start (pulmonary function tests), just say Jim, Mary, Tom, we have no idea where you’re at. Just give us your best shot. You know what I mean? A little bit of a pep talk probably wouldn’t hurt.'''

final_summary = recursive_summarize(text)
print("\n%%%%%%%%%%%%%%%%%%%%%\n")
print("Final summary:", final_summary)

######### Recursion level:  1 

######### 
Number of pieces:  3
****************************************************
Piece: 1  out of  3 pieces
I hope my participation in the [Pulmonary Fibrosis Foundation] Registry can ultimately help other patients understand and plan for what their disease may bring. How do you learn to live a fairly productive life when you have a progressive disease and there’s no playbook on what to expect? By filling out quality-of-life questionnaires for the Registry, I want to provide enough data points so that healthcare providers can help their patients work through their struggles.
Nevertheless, questions remained, and one of them was worrisome enough to get people to consider an unconventional approach to follow-up research. The problem is that the results that come out of drug studies in mice almost never translate to humans… As a result, we have close to thirty failed drug trials in pulmonary fibrosis alone.
I take part in every research study I can… May

In [ ]:
# Example usage - Patient experiences and needs related to evidence/translation
text = '''There’s 10 plus visits in a year … then at each visit, it’s at least a two-hour visit … that’s a fair amount of hard work as a patient. And I’m just not sure that I’d be up to that amount of work … I probably would still go forward with it … but having looked at what my requirements are for it, it’s just dampened my enthusiasm of it. I’m still going forward with it, I’m just not as excited.
Getting to the weekly visits, that can be difficult. My wife is in a wheelchair.
I suffer from chronic fatigue … Questionnaires are taken at every visit so that’s going to be a 30-minute function so I’m not sure that, that needs to be done frankly.
I think two hours would be probably the outer limit of length. I could do it, but I just think anything more than that is pushing it ... you’re going to be dealing with people who aren’t always having a good day.
There’s a lot of blood tests ... And when I saw this, I thought, wow, that’s a lot. And each time ... each time you get blood tests right for example.
I was wondering about that and the blood test frequency, a little bit high I would say. when I saw this, I thought, wow, that’s a lot. And each time ... each time you get blood tests right for example.
The total, the 52 weeks didn’t seem like a deal breaker but the greater the number I think the more people would find more onerous.
Chronic fatigue syndrome; I only have five to six hours a day of energy to complete activities … so for me … the day is short for me. And chronic fatigue is common amongst ILD patients, so we got to think about that.
Participants don’t have to change their medications or how often they see their physician.
If you have a big site … you may have to walk a mile … I can’t do a mile … I don’t typically have a wheelchair, but I might need one for that … it’d be nice to get a shuttle from the parking lot, or get dropped off at a door, or something to know where I’m going and how I could best get there.
Either bus passes, or city transit, or parking … or the city passes, or … Uber credits or something along that line is good, because it’s hard for some people to get around.
As long as they’re, I would say, probably morning to early afternoon [discussing visits] is better for most of the patients. His body starts shutting down probably by three in the afternoon. He starts
going downhill.
It would help that in working with the study coordinator, that they would have a little bit of flexibility. And if I would have a vacation planned [at] visit 6 that we could, we can move it out a week, but still be within that time period, and do something. So, we just have to see.
So for instance, I don’t know to change my time or the day of my visit, but as I say, I will do my best. I will do my best to stick to the visits that they will schedule for me.
I’m big on reminder emails … remind them of visits and remind them at this visit here’s what we’re going to do … just to set the expectation that it’s going to take a while. And then … if there’s anything that needs to be done beforehand. If you tell them up front … maybe that’s a little bit helpful.
If we had an agenda. If we had a schedule, I would feel less anguish. I would feel more safe.
I would do whatever I needed to do, but I just thought it was much more reasonable than having to travel far, and the fact that you can do some things on the computer. I liked that.
Yes, I would benefit indeed [from virtual visit option] because not all the days are the same.
I guess the most positive thing is that it can be done from home.
I prefer visits. Well, I prefer … the person I care for to undergo tests for better control, for better monitoring … But if visits were to be virtual, my scoring will not change.
Because everything that has to be done through the computer is more difficult for me, so I’d rather go to a place in person and get the tests done. I come from a different generation.
I would expect them to tell me potential adverse events … long term adverse events. I would like them to let me know about them … I always take into account whether it will be detrimental as well for me in terms of adverse events.
Maybe test results, anything to do with the test results in the trial … if the results could be sent to my PCP and be seen by my PCP as well, that would be great.
Well, I guess that my primary care physician would like to hear about the … regular testing that I will be undergoing throughout the clinical trial … he refers me to other doctors … therefore it’ll be very important for him to know about those tests, the outcomes, the results, how those tests were done.
Well the side effects, diarrhoea. I was wondering about that … that really stuck in my mind in terms of concerns.
[In the context of discussions about side effects] In principle, I’d like to know everything … I’d like to know if there has been secondary effects, I guess this has already been tried on humans. That’s what I’d like to know.
More than the primary care, I think the pulmonologist would be the one who should be informed. The specialist … I think I would be telling him that I’m getting a trial to see what his opinion is. Once I’m in, I would like for him to have the tests and results at his disposal.
It’s a really good to have that person [discussing study care navigator] that you know you can always get a hold of because sometimes the doctor … is not available. So having that contact there that is always available … I think to navigate everything and again takes some of that stress level off of you.
I think it’s all about mental health and I’m thinking about maybe a psychologist, psychological support, having some visits with a professional where she can really explain or share those concerns. I think it goes around the same idea of the peer group or being linked to this trial beyond just a medical fact, the fact that it’s personally and psychologically giving her something.
Well when you have a peer group you can feel more supported. Psychologically it’s really helpful at least for me as a caretaker and for my mom as a patient to be able to share her experience with someone who’s going through the same thing, I think that’s very helpful.
At least in studies that I’ve seen, I’ve never seen study partner caregiver compensation. I highly encourage it because they have costs, and they have skin in the game.
I’m just worrying if something like that happened, and he needs to be hospitalized, and he needed to be on more prednisone than he normally is ... I see that it’s not permitted.
No, the thing that worried me was, if he had to stop any of his meds that he was on now ... when I saw the list of meds, he wouldn’t have to stop any of his regular meds and be taking placebo. I was OK with that even if he was just taking placebo.
We went to different hospitals, and they said it’s a sequela of COVID and so we can’t do anything with that. And they don’t have any experience so we can’t do anything.
They say you are like this and that’s how you are going to end up like. And you think, what’s going to happen in between?
Well, I’m in the dark about what you can do to help the condition.
Lack of information always causes more worries.
If I was diagnosed with lung cancer it would have knocked me over, but knowing I have a fighting chance as things are, it’s not bothering me.
I was pleased it wasn’t cancer.
Doctors should be sensitive when delivering a diagnosis and prognosis to patients. The delivery of a diagnosis should not be rushed as the news is life changing. Time should not be an issue.
If somebody says to you it’s mild, moderate or severe, all that immediately goes through your mind is does that mean I have a short life or a long one? It doesn’t tell you what you’re faced with.
It took me years to get diagnosed and I know this can isolate you so much because you just feel you need to know what is wrong with you so you can deal with it.
It is extremely unsettling to have a disease that people say is rare and incurable. The research that I did on ILD was also hopeless pointing to low 5 and 10 year survival rates ... I had a lot of unanswered questions: How would my appearance change? Would I suffer? What other organs would be impacted after my lungs? What did I need to do to ensure the financial stability of my family? What did I need to do so that my family did not need me anymore?
The diagnosis was communicated to me in a realistic manner. There is no treatment and no cure but – I remember the doctor emphasised that but – but there is a medicine that gives encouraging results.
Being told you have this and there is nothing you can do is frightening. I was told that I wasn’t suitable for transplant, so I felt in a position of no hope. I just had to sit down and take what was coming with no hope, and that was the worst of all.
I was told my life expectancy was two to five years and then I was scheduled for a check-up for six months later. I was referred to a physiotherapist to learn breathing exercises. That was it – no other information or support was given.
More thought should be given to the psychological impact of being diagnosed with this devastating disease.
We were not given a lot of information. Most of what I learned was from attending a support group. I was told when the illness was diagnosed that it was terminal, which came as a shock to both me and my wife.
I didn’t know what to do initially ... I tried looking at the internet and the information that was available managed to confuse me even more.
I think at diagnosis, obviously, you need to be told that this is not curable and it’s progressive and it will end your life.
I think information can be provided in [the] way of a website, or even at the respiratory clinics, little leaflets for people to take. For example, having problems with exercise or keep healthy eating or, you know, when you’re feeling down, these are the resources.
[Information] should be available to you very early on, to put you onto the right track of exercise and diet.
I think that [what] the website should include, is that you’re not going to keel over from just exerting yourself a little bit. I was really frightened to do any exercise in case I made things worse.
The missing link for patients is the lack of information among the GPs. They are not informed about the disease and, in turn, they cannot properly advise patients or refer them to the right specialists.
We knew nothing about pulmonary fibrosis when mum received her diagnosis and immediately researched it. Google was not kind and we couldn’t believe something so cruel and terminal was unheard of. Anyone we spoke to about pulmonary fibrosis also hadn’t heard about pulmonary fibrosis and we soon became educators. This really needs to change. There has to be more awareness and support for pulmonary fibrosis sufferers and their families.
It was so hard to find information and any doctors or nurses we did talk to had never had a patient with IPF before. So what they maybe heard about or they were comparing it to COPD or other diseases that it really can’t be compared to. There was no information and it was really hard to find, and it felt like we were completely on our own. From what I found online, it was scary; but I figured if that was the case, then her doctor would tell us.
[In the context of advanced care planning and end-of-life care] I don’t think the GP knows enough about IPF to be honest because in my experience, they have heard of it but they might have one patient with it but it seems like by the time people are diagnosed, they are too far gone; or it just seems like it’s getting easier to diagnose ... Even the Palliative Care Nurse didn’t know about the oxygen testing and didn’t know about lung disease at all ... So I think they need to be more educated.'''

final_summary = recursive_summarize(text)
print("\n%%%%%%%%%%%%%%%%%%%%%\n")
print("Final summary:", final_summary)

######### Recursion level:  1 

######### 
Number of pieces:  16
****************************************************
Piece: 1  out of  16 pieces
There’s 10 plus visits in a year … then at each visit, it’s at least a two-hour visit … that’s a fair amount of hard work as a patient. And I’m just not sure that I’d be up to that amount of work … I probably would still go forward with it … but having looked at what my requirements are for it, it’s just dampened my enthusiasm of it. I’m still going forward with it, I’m just not as excited.
Getting to the weekly visits, that can be difficult. My wife is in a wheelchair.
I suffer from chronic fatigue … Questionnaires are taken at every visit so that’s going to be a 30-minute function so I’m not sure that, that needs to be done frankly.
I think two hours would be probably the outer limit of length. I could do it, but I just think anything more than 

SUMNMARY:  "I'm just not sure that I’d be up to that amount of work," he says. "I could do it, 

In [ ]:
# Example usage - Patients’ perspectives on design of clinical trials including patients’ experiences on trial site visits and logistics
text = '''There’s 10 plus visits in a year … then at each visit, it’s at least a two-hour visit … that’s a fair amount of hard work as a patient. And I’m just not sure that I’d be up to that amount of work … I probably would still go forward with it … but having looked at what my requirements are for it, it’s just dampened my enthusiasm of it. I’m still going forward with it, I’m just not as excited.
Getting to the weekly visits, that can be difficult. My wife is in a wheelchair.
I suffer from chronic fatigue … Questionnaires are taken at every visit so that’s going to be a 30-minute function so I’m not sure that, that needs to be done frankly.
I think two hours would be probably the outer limit of length. I could do it, but I just think anything more than that is pushing it ... you’re going to be dealing with people who aren’t always having a good day.
There’s a lot of blood tests ... And when I saw this, I thought, wow, that’s a lot. And each time ... each time you get blood tests right for example.
I was wondering about that and the blood test frequency, a little bit high I would say. when I saw this, I thought, wow, that’s a lot. And each time ... each time you get blood tests right for example.
The total, the 52 weeks didn’t seem like a deal breaker but the greater the number I think the more people would find more onerous.
Chronic fatigue syndrome; I only have five to six hours a day of energy to complete activities … so for me … the day is short for me. And chronic fatigue is common amongst ILD patients, so we got to think about that.
Participants don’t have to change their medications or how often they see their physician.
If you have a big site … you may have to walk a mile … I can’t do a mile … I don’t typically have a wheelchair, but I might need one for that … it’d be nice to get a shuttle from the parking lot, or get dropped off at a door, or something to know where I’m going and how I could best get there.
Either bus passes, or city transit, or parking … or the city passes, or … Uber credits or something along that line is good, because it’s hard for some people to get around.
As long as they’re, I would say, probably morning to early afternoon [discussing visits] is better for most of the patients. His body starts shutting down probably by three in the afternoon. He starts
going downhill.
It would help that in working with the study coordinator, that they would have a little bit of flexibility. And if I would have a vacation planned [at] visit 6 that we could, we can move it out a week, but still be within that time period, and do something. So, we just have to see.
So for instance, I don’t know to change my time or the day of my visit, but as I say, I will do my best. I will do my best to stick to the visits that they will schedule for me.
I’m big on reminder emails … remind them of visits and remind them at this visit here’s what we’re going to do … just to set the expectation that it’s going to take a while. And then … if there’s anything that needs to be done beforehand. If you tell them up front … maybe that’s a little bit helpful.
If we had an agenda. If we had a schedule, I would feel less anguish. I would feel more safe.
I would do whatever I needed to do, but I just thought it was much more reasonable than having to travel far, and the fact that you can do some things on the computer. I liked that.
Yes, I would benefit indeed [from virtual visit option] because not all the days are the same.
I guess the most positive thing is that it can be done from home.
I prefer visits. Well, I prefer … the person I care for to undergo tests for better control, for better monitoring … But if visits were to be virtual, my scoring will not change.
Because everything that has to be done through the computer is more difficult for me, so I’d rather go to a place in person and get the tests done. I come from a different generation.
I would expect them to tell me potential adverse events … long term adverse events. I would like them to let me know about them … I always take into account whether it will be detrimental as well for me in terms of adverse events.
Maybe test results, anything to do with the test results in the trial … if the results could be sent to my PCP and be seen by my PCP as well, that would be great.
Well, I guess that my primary care physician would like to hear about the … regular testing that I will be undergoing throughout the clinical trial … he refers me to other doctors … therefore it’ll be very important for him to know about those tests, the outcomes, the results, how those tests were done.
Well the side effects, diarrhoea. I was wondering about that … that really stuck in my mind in terms of concerns.
[In the context of discussions about side effects] In principle, I’d like to know everything … I’d like to know if there has been secondary effects, I guess this has already been tried on humans. That’s what I’d like to know.
More than the primary care, I think the pulmonologist would be the one who should be informed. The specialist … I think I would be telling him that I’m getting a trial to see what his opinion is. Once I’m in, I would like for him to have the tests and results at his disposal.
It’s a really good to have that person [discussing study care navigator] that you know you can always get a hold of because sometimes the doctor … is not available. So having that contact there that is always available … I think to navigate everything and again takes some of that stress level off of you.
I think it’s all about mental health and I’m thinking about maybe a psychologist, psychological support, having some visits with a professional where she can really explain or share those concerns. I think it goes around the same idea of the peer group or being linked to this trial beyond just a medical fact, the fact that it’s personally and psychologically giving her something.
Well when you have a peer group you can feel more supported. Psychologically it’s really helpful at least for me as a caretaker and for my mom as a patient to be able to share her experience with someone who’s going through the same thing, I think that’s very helpful.
At least in studies that I’ve seen, I’ve never seen study partner caregiver compensation. I highly encourage it because they have costs, and they have skin in the game.
I’m just worrying if something like that happened, and he needs to be hospitalized, and he needed to be on more prednisone than he normally is ... I see that it’s not permitted.
No, the thing that worried me was, if he had to stop any of his meds that he was on now ... when I saw the list of meds, he wouldn’t have to stop any of his regular meds and be taking placebo. I was OK with that even if he was just taking placebo.
We went to different hospitals, and they said it’s a sequela of COVID and so we can’t do anything with that. And they don’t have any experience so we can’t do anything.
They say you are like this and that’s how you are going to end up like. And you think, what’s going to happen in between?
Well, I’m in the dark about what you can do to help the condition.
Lack of information always causes more worries.
If I was diagnosed with lung cancer it would have knocked me over, but knowing I have a fighting chance as things are, it’s not bothering me.
I was pleased it wasn’t cancer.
Doctors should be sensitive when delivering a diagnosis and prognosis to patients. The delivery of a diagnosis should not be rushed as the news is life changing. Time should not be an issue.
If somebody says to you it’s mild, moderate or severe, all that immediately goes through your mind is does that mean I have a short life or a long one? It doesn’t tell you what you’re faced with.
It took me years to get diagnosed and I know this can isolate you so much because you just feel you need to know what is wrong with you so you can deal with it.
It is extremely unsettling to have a disease that people say is rare and incurable. The research that I did on ILD was also hopeless pointing to low 5 and 10 year survival rates ... I had a lot of unanswered questions: How would my appearance change? Would I suffer? What other organs would be impacted after my lungs? What did I need to do to ensure the financial stability of my family? What did I need to do so that my family did not need me anymore?
The diagnosis was communicated to me in a realistic manner. There is no treatment and no cure but – I remember the doctor emphasised that but – but there is a medicine that gives encouraging results.
Being told you have this and there is nothing you can do is frightening. I was told that I wasn’t suitable for transplant, so I felt in a position of no hope. I just had to sit down and take what was coming with no hope, and that was the worst of all.
I was told my life expectancy was two to five years and then I was scheduled for a check-up for six months later. I was referred to a physiotherapist to learn breathing exercises. That was it – no other information or support was given.
More thought should be given to the psychological impact of being diagnosed with this devastating disease.
We were not given a lot of information. Most of what I learned was from attending a support group. I was told when the illness was diagnosed that it was terminal, which came as a shock to both me and my wife.
I didn’t know what to do initially ... I tried looking at the internet and the information that was available managed to confuse me even more.
I think at diagnosis, obviously, you need to be told that this is not curable and it’s progressive and it will end your life.
I think information can be provided in [the] way of a website, or even at the respiratory clinics, little leaflets for people to take. For example, having problems with exercise or keep healthy eating or, you know, when you’re feeling down, these are the resources.
[Information] should be available to you very early on, to put you onto the right track of exercise and diet.
I think that [what] the website should include, is that you’re not going to keel over from just exerting yourself a little bit. I was really frightened to do any exercise in case I made things worse.
The missing link for patients is the lack of information among the GPs. They are not informed about the disease and, in turn, they cannot properly advise patients or refer them to the right specialists.
We knew nothing about pulmonary fibrosis when mum received her diagnosis and immediately researched it. Google was not kind and we couldn’t believe something so cruel and terminal was unheard of. Anyone we spoke to about pulmonary fibrosis also hadn’t heard about pulmonary fibrosis and we soon became educators. This really needs to change. There has to be more awareness and support for pulmonary fibrosis sufferers and their families.
It was so hard to find information and any doctors or nurses we did talk to had never had a patient with IPF before. So what they maybe heard about or they were comparing it to COPD or other diseases that it really can’t be compared to. There was no information and it was really hard to find, and it felt like we were completely on our own. From what I found online, it was scary; but I figured if that was the case, then her doctor would tell us.
[In the context of advanced care planning and end-of-life care] I don’t think the GP knows enough about IPF to be honest because in my experience, they have heard of it but they might have one patient with it but it seems like by the time people are diagnosed, they are too far gone; or it just seems like it’s getting easier to diagnose ... Even the Palliative Care Nurse didn’t know about the oxygen testing and didn’t know about lung disease at all ... So I think they need to be more educated.'''

final_summary = recursive_summarize(text)
print("\n%%%%%%%%%%%%%%%%%%%%%\n")
print("Final summary:", final_summary)

######### Recursion level:  1 

######### 
Number of pieces:  16
****************************************************
Piece: 1  out of  16 pieces
There’s 10 plus visits in a year … then at each visit, it’s at least a two-hour visit … that’s a fair amount of hard work as a patient. And I’m just not sure that I’d be up to that amount of work … I probably would still go forward with it … but having looked at what my requirements are for it, it’s just dampened my enthusiasm of it. I’m still going forward with it, I’m just not as excited.
Getting to the weekly visits, that can be difficult. My wife is in a wheelchair.
I suffer from chronic fatigue … Questionnaires are taken at every visit so that’s going to be a 30-minute function so I’m not sure that, that needs to be done frankly.
I think two hours would be probably the outer limit of length. I could do it, but I just think anything more than 

SUMNMARY:  "I'm just not sure that I’d be up to that amount of work," he says. "I could do it, 

In [ ]:
# Example usage - Patient experience and needs related to drug and delivery
text = '''Keep working with the doctors. I think a lot of the issues in the medical community is that it is still kind of an unknown disease. Especially in those smaller centers, where maybe the doctor has never seen this.
I run everything through my GP first. So, once he found out that I have this, he did not know about it. And he told me, I’ve never had a patient like this before. I’m going to learn all about this. And he spent the time learning. He learned on the journey with me, so that’s pretty cool of my doctor.
I believe it is important that healthcare professionals are honest with patients and families about the risks, so they can base their decisions on the facts.
The support my father got was very good, especially from the specialist nurse who helped him to manage his condition.
I have a good relationship with both the nurse and the respiratory therapist. They’ll decide on a plan, and then the nurse communicates with [the ILD physician] or [the rehabilitation/palliative respiratory care physician]. They handle all of that really well. We [patient and the nurse] had a conversation around this ... about how well our relationship is working. And I trust her [the nurse] implicitly to be communicative with [the ILD physician and the rehabilitation/ palliative respiratory care physician].
You have full involvement and knowledge in what’s going on. It’s very clear to us she’s [the ILD physician] going to see us through to the end of this and it’s her primary concern to do the best for us that she can. And that’s a very good feeling—a feeling of trust and security. We’re just feeling very lucky to have somebody like her and her entire group because they all have that same approach or philosophy to it all.
We were terrified to have those conversations but once we knew for sure that was what was happening, we made it a point to talk about it openly and talk about her fears and concerns and that was huge. That was one thing that Dr. J nailed was being up front. What do you want your death to look like? How do you want to die? What scares you the most about it? And for my mom it was—she didn’t want to be drugged. She wanted to be conscious right until she couldn’t be anymore so they kept that in mind.
Do I just go into hospital and I just gasp until I die? But, being reassured by them that I can have a home death, there can be drugs that can help to calm me down, and people there ... Yes. And how caring and supportive the doctors can be. That there are drugs, medications that they can give you to calm you down, so it’s not scary at the end.
I ended up in hospital for a week on IV antibiotics and a chest tube. Plus they produced a Green Sleeve, which I’d never seen or heard about until then. I can see their point. They want to get it done before you end up needing resuscitation. But honest to god if you’re short of breath, got pains in your side and feeling lousy, it’s not necessarily the time to be asking you questions about levels of care.
And where you want to die and what the different types of experiences look like—so dying in a hospital versus a hospice versus at home. And with each of those different scenarios— the positives and negatives about it ... So it is really just about having those conversations and so people know what to expect and they are not blind-sided by it and making sure that the hospice is willing to work with them. And if they are going to stay at home, what resources are available to them?
I think everything should be addressed … There’s so many facets to a terminal disease that you don’t even think about before you’re diagnosed ... It’s really hard ... By being honest. By laying it on the line and telling me this is what we do. This is what we can do. This is what could happen ... It’s very important. It’s one of my biggest fears. But now that I know a bit more about it, I go to a support group that Dr. J spoke about ... and I do feel much better just knowing. I was afraid of the end. Of how violent it could be. From things that I’ve heard … it was very calm and very reassuring.
My hope is that oxygen companies will realize that oxygen needs are not a one size fits all across patients and will work more closely and compassionately with patients to ensure their oxygen needs can be met in a way that allows as much quality of life and as little frustration as possible.
I know that since my transplant there has been great strides with new drugs to slow down the progression of IPF and research continues to find a cure. I only hope and pray that this will be made possible soon so the need for transplantation will lessen.
There was a general lack of understanding of the condition’s progression by GPs and others, and no services available to support patients and their families at the end of life.
In my view, the health system let my mother, and us as a family, down badly at the end of her life, due to disjointed services, breakdowns in communication, and the fact that there was no one person to coordinate health and care services. It was truly a traumatic experience.
I do feel that unlike cancer patients, pulmonary fibrosis sufferers do not receive the same emotional and mental support to help prepare them through the journey ... Explaining pulmonary fibrosis to anyone was a challenge… This really needs to change as it’s deeply unfair and seems borderline discriminatory. Anyone going through pulmonary fibrosis needs to be supported and understood, just like any other terminal illness. Getting any support for mum was hard because it felt like her condition was not taken seriously until it became dire.
My father died of pulmonary fibrosis when I was in college, and that was more than twenty years ago. In all those years, there’s been no change in a patient’s experience with the disease.'''

final_summary = recursive_summarize(text)
print("\n%%%%%%%%%%%%%%%%%%%%%\n")
print("Final summary:", final_summary)

######### Recursion level:  1 

######### 
Number of pieces:  8
****************************************************
Piece: 1  out of  8 pieces
Keep working with the doctors. I think a lot of the issues in the medical community is that it is still kind of an unknown disease. Especially in those smaller centers, where maybe the doctor has never seen this.
I run everything through my GP first. So, once he found out that I have this, he did not know about it. And he told me, I’ve never had a patient like this before. I’m going to learn all about this. And he spent the time learning. He learned on the journey with me, so that’s pretty cool of my doctor.
I believe it is important that healthcare professionals are honest with patients and families about the risks, so they can base their decisions on the facts.
The support my father got was very good, especially from the specialist nurse who helped him to manage his condition.
I have a good relationship with both the nurse and the respiratory

In [ ]:
# Example usage - Patient experience and needs related to drug and delivery
text = '''### Living with IPF ###

I was diagnosed with Idiopathic Pulmonary Fibrosis, also known as IPF. I was only 28 years old.
IPF has been a surprise and a life-altering challenge. I’m nearly 71 years old, and was formally diagnosed three years ago.
At the age of 4, I was diagnosed with juvenile rheumatoid arthritis… we counted ourselves lucky that it appeared I had the presentation where it attacks primarily the joints. We didn’t realize this would later take our journey down another path… My childhood had its aches and pains. After college was when things really started to change. I was diagnosed with double pneumonia shortly after graduating… Blood tests, CT scans and a lung biopsy eventually confirmed a diagnosis of pulmonary fibrosis in 2011. My family and I later found out that rheumatoid arthritis caused my pulmonary fibrosis.

### Symptoms, comorbidities and physical/cognitive limitations ###

Breathlessness is really difficult to describe. I can only liken it to feeling like you’re under water and want to breathe, but you can’t. I’ve had to learn techniques to help me manage my breathlessness.
[Coughing] can cause me to vomit ... that impacts on myself and my family, keeping us awake at night particularly.
Feelings of weakness and breathlessness overtake the enjoyment and ability to do much.
I found myself persistently short of breath, fatigued, and dealing with a cough that wouldn’t subside.
Coughing all the time is exhausting. I have little energy to do as I like, like spending time with my grandchildren.
Patients also report weight loss as a symptom of PF:
Weight loss seems to be part and parcel of IPF and mine has dropped from 90kg to about 67kg now, so you have to eat what you need to maintain health.
That’s been progressive and weight loss ... and very low BMI and advanced nutritional support.
If it’s really cold outside… it hurts me to go outside. It really hurts my lungs. Heat and humidity really, really take their toll. Allergy season is crazy.
Patients’ also describe the progressive worsening of symptoms over time:
I first noticed my breathing problems on a long-haul flight and struggled with my breathing while I was away. I thought it was a virus, but after some planned surgery the following month I developed a cough, struggled with my breathing and was admitted to the emergency department ... At times I’d feel fine, then going upstairs would leave me incredibly breathless. I developed a persistent cough and felt tired a lot of the time.
As the disease progressed the coughing became more prevalent. It affected simple day acts; I’d cough when I laughed, after eating and in atmospheres where there are smells of chemicals, perfumes and aerosol deodorants, or spaces too hot or cold. The cough has increased and sounds different as the time passes. The sound of my cough is more grave and I now cough for longer periods.
Flare-up of symptoms is reported to be severely debilitating:
In May 2018 I had a flare-up of my symptoms while in Rome due to a respiratory infection. I was coughing all day and night, and couldn’t sleep. It took about 8 weeks and some pretty strong antibiotics to shift it completely.
I have a constant dry cough. Very annoying. I am on supplemental oxygen 24 hours a day. I even have canisters that I use when I go outside of the house. But it’s a pain... I have some days where it’s not really that bad. I have some days where it is so severe that it’s – I cough to the point that I throw up because I’m coughing so much.
My symptoms were heavy labored breathing when I did normal tasks such as climbing a flight of stairs and numbness and discoloration in my toes and fingers or Raynaud’s syndrome.
I feel like the last couple of months I’ve been feeling more fatigue than I have in the past… Sometimes there [are] things I have to do, so I kind of work through it, but, you know, when I, when I don’t lie down and I still go to work and still do things, it drains me even more.
Sometimes I get chest pain. It comes and goes, but it mostly feels like somebody really heavy, like I feel a 400-pound person, like I got an extra body sitting on me.
I couldn’t walk along freely without oxygen. I had to have somebody nearby just in case, because I have these dizzy spells and sometimes I would pass out.
It’s just one of those feelings where [it’s] kind of like you panic. You know, you want air in. Because you need air to breathe, and if you don’t have air you just feel all discombobulated. That’s kind of a weird feeling.
I was diagnosed in 2007 with Pulmonary Fibrosis because of Lupus an autoimmune disease which caused the damage to my lungs.
Patient with PF caused by antibiotics for urinary tract infection:
I got the Fibrosis, which is life shortening, by repeatedly being prescribed an antibiotic for urinary tract infection. The antibiotic is Nitrofurantoin.
Patients with PF caused by taking medications for their atrial fibrillation:
I have pulmonary fibrosis caused by the treatment I had for AF. I had amiodarone, which is known to cause lung problems, and then dronedarone, which is safer. Unfortunately, I developed pulmonary fibrosis.
I too have pulmonary fibrosis, diagnosed 4 years ago and caused by the drug I was taking for my atrial fibrillation, which was then stopped, but the damage was done. I also have other health problems with my heart and lungs.
When I initially started, I was itching every day, and it would keep me up at night. It was just constantly itching.
At the beginning, when I was first diagnosed, I used to be a really, severe Raynaud’s where just touching the fingers were painful.
I also have severe osteoarthritis in both knees.
Where now, I don’t have that sensitivity in the fingertips. It will turn purple; it will hurt but not as bad as in the beginning when I was diagnosed. I used to get sores-I used to get digital ulcers and scars and sores, and now I don’t. Fingers turn blue but I don’t get the sores.
I have Raynaud’s, my fingers are curling up and I have lost my nails, pulmonary fibrosis along with the scleroderma, acid reflux, aspergillosis and an irregular heart.

### Diagnosis, referral and treatment pathways ###
I started experiencing a buildup of mucus in my throat causing a constant clearing and coughing. The colour was always clear so, since both my family doctor and I felt it was post nasal drip, I was prescribed nasal sprays.
Before my diagnosis I’d been feeling some pain on my lower back that wouldn’t go away. I also noticed that I often needed to cough to clear my throat before having a conversation but I wouldn’t call it severe.
It took around 2 weeks for her to be diagnosed after being taken to A&E at Hammersmith Hospital, London when she felt her breathing was not right. I remember being told at the time that her lungs were too weak for a biopsy to be done to confirm her diagnosis. Shortly after, she was given a prognosis of 5 years. This was a massive shock and we felt totally winded. It was unexplainable for us and felt surreal.
I first mentioned my cough to my GP at my annual health check in 2013. The X-ray came back clear. But a year later, another X-ray showed something on my left lung. Lots of tests were inconclusive, but finally the results of a surgical lung biopsy indicated I had IPF.
It took 2 years of back and forth to get to this stage!
Then we moved and I saw a new GP. I was sent for a CT scan almost immediately, which showed that there was damage in my airways. I was told I had pulmonary fibrosis and referred to hospital. After a lung biopsy and further tests I was finally diagnosed, three years after my first visit to the GP.
At the age of 4, I was diagnosed with juvenile rheumatoid arthritis… we counted ourselves lucky that it appeared I had the presentation where it attacks primarily the joints. We didn’t realize this would later take our journey down another path… My childhood had its aches and pains. After college was when things really started to change. I was diagnosed with double pneumonia shortly after graduating… Blood tests, CT scans and a lung biopsy eventually confirmed a diagnosis of pulmonary fibrosis in 2011. My family and I later found out that rheumatoid arthritis caused my pulmonary fibrosis.
I was diagnosed with double pneumonia shortly after graduating. After recovering, I still had a cough that I couldn’t shake no matter how many over-the-counter or prescription edications I tried, and while I was far from an athlete before, the shortness of breath was slightly more noticeable now.
At first I was told I had arthritis, then rheumatoid arthritis. Lupus was also diagnosed and I was given treatments for them all. Nothing helped and I only got worse ... he ran a myriad of tests; X- rays, blood and urine ... and told us that I had scleroderma with CREST and pulmonary fibrosis.
My breathing complications became really bad so my chest specialist thought that I had a condition called bronchiectasis and did [an] MRI scan to detect it. This came back clear of bronchiectasis but he said we did find pulmonary fibrosis on the scan.

### Misdiagnosis ###

My dad was misdiagnosed for five years and was finally told he had IPF one week before he died.
I changed respirologists part way through (the process of being treated) because I wasn’t getting the information I wanted.
Doctor (diagnoses) with fibermyalgia [sic] and prescribed prednisone 40mg… My arthritis results came back negative… Upon visiting a physician he noticed that my oxygen is very very low… had CT scan with contrast of my entire chest where they discovered fibrosis.
I was first diagnosed with bronchiectasis in 2002. It took till 2011 before I discovered that I had pulmonary fibrosis.
I got a really bad respiratory flu. I didn’t recover in 10 or 12 days… and at first they thought it was COPD.
My health had been deteriorating for quite a while running up to my IPF diagnosis in 2015. In early 2014, I had been experiencing severe chest pains. An x-ray revealed honeycombing on my lungs and my doctor could hear crackling. Following a lung biopsy at hospital, my partner and I were initially told that I did not have IPF – we left the hospital feeling relieved. Unfortunately, the relief was short-lived. At my next routine appointment, another doctor informed me that I did have IPF. This came as a massive shock and I left the hospital feeling unhappy and anxious about the future. When I got home, I told Allison, who was distraught; having previously read that the average life expectancy from diagnosis was 3-5 years.
Very shocked. Really upset at the time. Coming to terms with it slowly.
They gave me a booklet and I wish I didn’t read it.
To give it to someone as soon as they diagnosed it, I think it was not a good idea.
While the diagnosis itself might be clearly explained at the time, patients report a subsequent lack of support and information:
After diagnosis, I have been left on my own to cope, with no help from anyone.
Things were very confusing at this time for me, and I remember wondering why I was being admitted, because I didn’t feel particularly worse than I had in the months prior.
I felt desperate and alone. After such an awful diagnosis, my next appointment was in 6 months and nothing in between.
To be diagnosed with IPF is absolutely devastating ... You are told you have three to five years to live ... It’s a very, very scary proposition.
I guess the reason why I felt so surprised was that there was no mention of pulmonary fibrosis in the years prior, when I had multiple episodes of flu, bronchitis, pneumonia.
It was a bit of a shock to the system ... we thought it was just a chest infection ... he had a little cough but nothing too much.
Initially it was a huge shock to discover that he had IPF, which I had never heard of.
Given that I am a life-long non-smoker and never knowingly worked in a hazardous environment, I couldn’t fathom my diagnosis. Why me and IPF?
I feel picked on, I feel I haven’t smoked for all these years, I don’t drink, I’m not overly fat, why is this happening to me?
My general practitioner had no idea what was wrong with me. I used to feel upset when I left his office as I knew he did not believe what I was telling him.
I was diagnosed with Interstitial Lung Disease (ILD) caused by scleroderma at the end of October in 2012… My first reaction was disbelief especially when I was told that one of the ‘treatments’ would be chemotherapy.
Blood tests, CT scans and a lung biopsy eventually confirmed a diagnosis of pulmonary fibrosis in 2011. My family and I later found out that rheumatoid arthritis caused my pulmonary fibrosis. I was shocked, but I was determined to see the bright side of things. We had a diagnosis and now could start on a path of monitoring this to hopefully prevent any further complications.
At the time of diagnosis I had no symptoms, I was athletic and completely healthy. I had no idea the impact it would come to have on my life.
I was told I had Interstitial Lung Disease and Pneumonitis which resulted in scarring and a partially collapsed left lung in early 2019 when I had been on MTX. Is that pulmonary fibrosis? I don’t know. I’m about to be referred to a Respiratory consultant (again) as I have a blood clot in each lung and so will ask for an explanation of all the different terms.
… but had always been diagnosed as something else, such as pneumonia. This unfortunately led to countless, useless prescriptions of antibiotics; to post-nasal drip; to possibly tuberculosis (simple tests proved otherwise). Luckily, my new and astute physician decided that the problem was greater and a referral to a respirologist confirmed IPF.
I was referred to a respirologist and put on a high dose steroid, and a bronchodilator inhaler. Several tests followed and finally in December 2010, based on a lung biopsy, I was diagnosed with IPF that had already reduced my lung function by 40%.
Have [sic] been diagnosed this by my respiratory consultant due to decrease in my lung function tests.
I was diagnosed with pulmonary fibrosis approx 5 years ago. My respiratory consultant believes it RA related and I’ve no reason to doubt him as I’m as sure as I can be that my lungs were damaged during a prolonged RA flare.
Am going to a connective tissue consultant… as my rheumatologist consultant thinks this would help me better.
My hospital has a scleroderma specialist, thankfully! I had been treated for pulmonary fibrosis there. Some other issues landed me with rheumatology, but they seem to have a few multidisciplinary specialists that see scleroderma patients, which is great.

### Treatment experience and patient role in treatment selection ####

I was diagnosed with pulmonary fibrosis in 2004 and subsequently got Pulmonary Hypertension as well. A year ago I was prescribed Hydroxychloroquine Sulfate.
I’m on a drug called nintedanib which I’ve been on for 4 years. I had to have the dose lowered to reduce side effects. We’re both also on long-term oxygen therapy.
I’ve been on nintedanib for nearly a year now. The drug isn’t expected to improve things as scarring can’t be reversed but it is hoped it will stop or slow down progression. I did feel better straight away but I am also taking steroids.
Ambulatory oxygen is very important – it helps me keep active.
I’ve recently been given an Inogen one oxygen concentrator. It only gives oxygen when you breathe through the tubes up your nose.
Patients describe having a strong support system in place as being important:
Eventually you become dependent on supplementary oxygen for exercise initially and then 24/7 at home just to stay alive.
You’re using your own supports whether it’s your friends, my telephones been burnt out. It’s just staying in touch with people, that’s kind of what gets you through the day.
Yoga, meditation, just going out for a walk, I mean all of those things definitely helped.
I started with the garden benches [home improvement project]. It’s just something to keep you going... keep the mind busy on something else… The more you sit around watching the television, the more things will get into your mind.
Patient groups also provide a powerful coping mechanism:
There was company, people with the same disease. I found I was looking forward to going. And I thought it was doing me good.
Just having a chat about families and how they are coping, you know that’s the biggest part of it. Getting together and talking to people with the same problem.
However, the influence was not consistently positive:
‘I couldn’t help thinking that [the patient group meeting] is like
[Dr Harold Shipman’s] waiting room, waiting for each one to pop off you know. Because there is always news that somebody is gone and that’s a person you know and I think, my name would be there one day wouldn’t it?’ (Dr Harold Shipman was a UK physician convicted of euthanasia of elderly patients)
 ... [There was a negative] down turn because you all know that you are going sooner than you’d like to be.
It’s sort of monitoring yourself and knowing what your capacity is, and if it’s… you’re getting to a… sort of a point where you need to take a break, well take a break you know, that’s all. I guess you learn yourself where those points are.
I think by, referring how it affects me to somebody else, may not help them at all, yet it helps me. And that’s why I’m saying, I think it’s sort of very individual.
That means that I have to be aware, more aware… and try and make sure [I] eat properly, exercise, monitor my oxygen requirements.
I have to, just get into a routine and acceptance of that, and find the little pockets of positivity within that well.
Preparing oneself for… acceptance of the illness, not just acceptance of the illness, but acceptance of the illness that where it’s going to take you, as in, in the end, you will die. But um, you can’t just put that out of your mind.
I know they (family) care and I know they’re gonna be there for me and that’s a big, a big plus, whether it’s on an emotional level or a physical level, they would do anything to help me that they could.
I can now restore my breathing by remaining as calm as possible and inhaling through my nose. […] I find that drinking water and keeping hydrated, having a small, low sugar sweet in my mouth (liquorice is quite effective) and having small meals, help me to reduce my cough.
I do find that my mouth dries out so I must do some mouth breathing, cause I’m sort of constantly putting on, you know, lip balm during the day, and at nighttime too.
My mother died of lung disease, she had fibrosis, so she had scarring all over her lungs and eventually it covered her lung and I mean, there’s no way you can breathe after a while, and I know this is mind over matter, but do you know those little propeller handheld fans? I mean, this is just mind over matter for sure, but that feeling of air just on her face calmed her down.
The decision to start treatment was mine, but it was based on my lung specialist’s advice because I am a layman.
I made the decision. I had to make my physician aware of the treatments to get them.
I asked the ILD specialist about which drug he would recommend, I was recommended Esbriet. However, I did NOT want to take a treatment that required me taking 9 tablets a day on top of all my heart medication that I already take.
My pulmonary fibrosis is due to Rheumatoid Arthritis [RA], the RA is being actively treated but constantly getting pneumonia requiring hospital admissions.
A low dose of Baracitinib for my RA. Apparently some rheumatologists don’t like prescribing baricitinib to people with lung conditions but I’ve been taking it for approx. 2 years without any detrimental effect on my lungs so far. My respiratory consultant advised me to stop taking methotrexate which I was on when diagnosed. Rheumy have stopped me taking all other DMARDs too, which I think is because of my pulmonary fibrosis.
The lack of multidisciplinary team plays a role here. For instance, physiotherapy is not contemplated as part of the process. The multidisciplinary team should not only include lung specialists, radiologists and pathologists, it should also include physiotherapists.
For a few months, I’ve been using ambulatory oxygen. I had struggled with the concept, as I saw it as a sign my condition was progressing. But now I see it as a way of allowing me to continue to do the things I want to do and as an important safeguard.
[Being on oxygen] enables you to carry on doing the things that you want to do… at least it enables you to go out and do some exercise… go off to the park, go and see friends… I think the way people think of oxygen needs to change… that it’s a positive thing.
I didn’t want to go on oxygen because it was accepting or confirming that I was getting worse. When I look back on it, I should have been on supplementary oxygen 6 months or more before.
To slow down the scarring of my lungs, I was prescribed pirfenidone. Initially I had a lot of side effects: nausea, vomiting, diarrhoea and a skin rash which meant I had to put on SPF50 sunscreen whenever I went out in the sun. I also felt very fatigued sometimes and would need to take short breaks during the day. But I tolerated all this as it was slowing the rate of progression of the scarring on my lungs.
Corticosteroids made me feel like a balloon.
Going out is always a challenge because I worry about running out of oxygen.
The three most common challenges I’ve faced are travelling with oxygen, getting the appropriate oxygen for home use, and exercising with oxygen.
I was amazed at how useful it was; it told me things I feel my GP should have; things I needed to know. It gave me the confidence to go to the gym, even with my oxygen. I met a friend on the course and we went together and both exercised with our oxygen.
At the course I found out breathing techniques that I still use today, years later. It helped me to cope and accept my condition.
I did pulmonary rehab and it really helped. But because I’ve already done it, and there’s such a waiting list, I haven’t gone back.
The doctor suggested I go again recently, and I said to the doctor I don’t really feel I can do it. Or have the energy. And then my husband would have to drive me, and it’s just sheer distance.
What really helped me was going to a pulmonary rehab program. That was a way of being able to start getting back into exercising, and activity, and wellness exercises, and connecting with other people who have lung issues.
The fitter your body is, the better you feel both physically and mentally. If you feel better mentally then your whole well-being is more positive and you won’t dwell on the negative sides of the disease.
Palliative care can be very useful… you can get medication… assistance, help with nutritional changes, and also participate in relaxation techniques, and get advice on social, emotional and spiritual… Psychological support is very, very important.
I often forget to take the one pill I have to take weekly.
I prefer to experience diarrhoea than a headache as my line of work requires lots of concentration. I tend to gain weight — specifically, corticosteroids made me feel like a balloon. I believe that at the end of the day… any side effects worsen quality of life.
If they are all severe, diarrhoea is the hardest… every moment of the day going to the toilet can be embarrassing and hits that physical and pride level.
The most common thing that a lot of people can’t manage is the diarrhoea, because you’re literally at home and you can’t go anywhere.
The sun sensitivity is especially inconvenient as my wife comes from the Caribbean and still wants to travel there and then it would be severe to travel there with such a risk of side effects.
I’m already on a cocktail of pills and taking nine more a day is too much to ask.
However, in June 2017, test results indicated that the medication wasn’t slowing the scarring as much as was hoped. So, I was switched to the only other available and relatively new drug, nintedanib. I was a bit nauseous initially, but I’ve found a good regime and manage my diet.
I was prescribed Esbriet at first but experienced intolerable side effects so the drug was exchanged for OFEV. Is it working? Well, my spirometry test shows little minimal change in the last 6 months.
Some people like me don’t can’t seem to absorb or respond to the Zentevia brand and only do well with the Bristol Brand/Quinoric Brand… Mine was switched in July and I landed up in hospital for 6 weeks. I never dreamt that it could be due to a medication brand switch but it was.
There are only 2 approved medications to help slow the condition and I have been on both – the first, Pirfenidone, was not as effective as hoped and so I was switched to Nintedanib.
Stephen preferred to have diarrhoea and only 2 pills a day instead of vomiting and 9 pills.
Oh, over the phone is perfect for me because I don’t want to be going in there at all… the hospital is the worst place in the world to be in these days.
There was great support in the knowledge that if I was unwell I could ring… That message was re-enforced many times, that if I needed a bit of help, someone to talk to.
Negative
You really do miss those (faceto-face) meetings… when you come away from it, it’s absolutely brilliant, it’s just the confidence.
I’d be praying night and day that there would not be any changes in my condition… I’d be often worried… I would not be able to self-diagnose myself.
For example, if you go to physio, they tell you how to do things [exercises] and then by the time you get home you’ve forgotten it. Whereas, if you’ve got a video, you can always fall back on it and make sure you are doing it correctly.
When you go to the doctors you get like seven minutes, the seven minutes finish and I have forgotten, you know what I mean. If you had something on the internet, you could watch it over and over an explain it, you know what I mean, so you can understand it.

I’ve often wondered to what extent the diagnosis is exactly correct. When I go online and read about my symptoms, I often find that the same symptoms apply to several diseases under the same umbrella. I’m diagnosed with IPF… some of the treatments seem to be a little bit different. It would be probably quite helpful if there were a genetic test that would say, ‘yeah, this is what you have, not that. And so your treatment should be (this).
It would be a real advantage if there was genetic testing developed that might catch something like pulmonary fibrosis right at the very beginning and start treatment right away instead of having it develop and cause lots of scarring before it’s diagnosed.
I think it would depend on how invasive it is and how accurate it is. I wouldn’t want it to leave me with more questions that there are no answers for.
I have siblings, one issue I think is whether … somebody would actually want to know… Some people would want to live their life accordingly. Others, if they think they’re going to get this terrible disease, they’ll be devastated, and it would really negatively affect their quality of life.
The sun sensitivity is especially inconvenient as my wife comes from the Caribbean and still wants to travel there and then it would be severe to travel there with such a risk of side effects.
However, in June 2017, test results indicated that the medication wasn’t slowing the scarring as much as was hoped. So I was switched to the only other available and relatively new drug, nintedanib. I was a bit nauseous initially, but I’ve found a good regime and manage my diet.
I was prescribed Esbriet at first but experienced intolerable side effects so the drug was exchanged for OFEV. Is it working? Well, my spirometry test shows little minimal change in the last 6 months.
Some people like me don’t can’t [sic] seem to absorb or respond to the Zentevia brand and only do well with the Bristol Brand/Quinoric Brand … Mine [medication] was switched in July and I landed up in hospital for 6 weeks. I never dreamt that it could be due to a medication brand switch but it was.
There are only 2 approved medications to help slow the condition and I have been on both – the first, Pirfenidone, was not as effective as hoped and so I was switched to Nintedanib.
Stephen [the patient] preferred to have diarrhoea and only 2 pills a day instead of vomiting and 9 pills.

### Specific impacts of PF on activities of daily living including navigating healthcare environment and access ###

As IPF was destroying my dad’s life, he struggled with the disappointment that no cure existed, and he looked for a way to support research that could lead to a cure.
‘I just wish it would stop. The feeling that it is never going to improve is sometimes overwhelming.
I used to play a lot of golf, swimming, cricket, you name it. I’ve had no problem, but now I’m a television viewer.
I’ve sometimes found lifting whatever across the garden these days is difficult, but before it was easy.
I try to keep everything in precise order, so things at the same place all the time.
It’s sort of reorganizing your life, try and do everything in one place before you move to another.
There are occasions when I can’t participate in events because I’m too fatigued and I’ve learned that it’s important to pay attention to my body when it’s telling me to rest ... I am determined not to let this disease steal my life… … The pain and distressing fatigue made it difficult to exercise, as well, which led to decreased function and more weight gain. I felt trapped in a distressing cycle of illness ... The damage to my lungs from the sarcoidosis has left me with many limitations but I try not to allow these limitations to stop me from living my best life. I work full- time, I travel, I exercise and interact with my friends and family.
I am on oxygen all the time, and can only walk a short distance. Because my left lung collapsed due to so much scar tissue, I have partially lost my voice.
I limit my physical activities, take measures to keep my hands and feet warm at all times, avoid unnecessary physical contact with people, and avoid eating out. I also try to limit my travel.
When I was first diagnosed [with IPF], we made significant lifestyle changes, we downsized our house in preparation and immediately booked all the holidays we could. I now only work part time, which enables me to work around the many hospital appointments. Allison accompanies me on most appointments for moral support and to listen to the detail.
Under occupational health advice, Charlie’s employer has altered his role so he is office based now and has reduced working hours.
I find it very frustrating that I cannot do the amount of manual tasks I used to do and quite often have to leave jobs unfinished for another time. Jobs I would previously do without thinking now have to be planned and timed. Some tasks I just can’t do at all anymore which goes against a lifetime of independence.
Daily routines like taking a shower, brushing my teeth, or getting dressed are a full time job.
As the disease progressed, it became more exhausting. I couldn’t do anything without oxygen. Everything I try to do, which would usually take a day to do, takes me several days instead.
I started to notice the effects of the disease and how difficult very simple tasks such as brushing my teeth, taking a shower, putting on clothes, chronic coughing with clear mucus, walking, and climbing stairs or inclines had become.
I get breathless doing everyday things like making a cup of tea. It can be quite isolating, but you’ve got to keep going and learn how to adapt to your new norm.
Sheetal became breathless when on a long walk or climbing stairs, but she was able to continue working, do the shopping, daily chores and look after Naisha. She wanted to live and be there for Naisha and was able to live a fairly normal life with her symptoms. She was a very strong and courageous person, in fact, the bravest woman I’ve known. Nothing phased her. But it must have been incredibly difficult for her living in the knowledge that she had limited time– particularly as a mother who wanted to be there to see her daughter grow up.
You effectively become disabled in your own house. You need care. You need to have somebody to help you with simple tasks like washing and dressing.
My breathing became laboured and simply walking exhausted me — stairs, hills and steep grades were killers. Basic chores involving any form of exertion not only slowed my ability to do things but my desire to do them.
I am on oxygen 24/7 but I still struggle with day-to-day activities.
Initially, you notice it because you find, you know, walking up hills difficult… then over time even walking up the stairs ... slowly it becomes difficult even to walk on the flat.
My activities are severely limited now. Going and taking in my recycling bin requires supplementary oxygen.
I can’t get out to get the paper every day, so I read the paper every morning, virtually.
I took time off from work and worked from home on many occasions.
I began to feel isolated and ostracized because of the cough. I was embarrassed by the shortness of breath. I was irritable and suffered from insomnia because of the Prednisone that I was on to control my symptoms, and I was depressed about the weight it caused me to gain.
Well, if you can’t breathe, you can’t do anything. And at this point of the journey, that’s about where I am. I can do no housework, I can’t even wash my hair. I have to plan everything that I’m going to do that’s going to take energy.
I had scarring after severe ARDS – it’s nickname is concrete lung – the lung becomes so rigid. The medical staff presumed I would lose a great deal of lung capacity. After some brilliant hospital physiotherapy, a year after ICU, my lungs were fairly normal. Unfortunately a pleural effusion of my left lung through having my leukaemia colonise this area, has left me slightly breathless but that was 2yrs later.
I am on hospice care now and have found it to be so helpful and encouraging. I am 77, but mostly enjoy life.
We adapted to a slower pace of life. Changes were made to our home to make it easier for us both to move around. We got used to a different social life, as I was unable to leave Peter for more than 2 hours at a time.
I try not to do things that trigger my cough, which is everything. I’m embarrassed with other people’s reactions as they don’t understand my condition.
[In the context of the COVID-19 pandemic] all that year, I had been pulling away from social interaction… So when March 2020 came along it was like more of the same.
… I’m trying to fill my time and concentrate on something else worthwhile, the rugby club and the APF charity fit the bill!
We’ve also become involved with the APF support group which is great for connecting with and helping others.
 Because who wants to go out, first thing they think is that I’m contagious, and I’m coughing all over the place [… ] the first thing people think is why can’t she do something about that cough? ... But they don’t understand what the problem is, why I can’t stop coughing, and it does, you know, I’m not going to say I get embarrassed by it, but I don’t want anybody to feel [un]comfortable [… ] You know, like recently there was something, my granddaughter received an award, and I couldn’t go. So I just made the choice not to go. But it has impacted my life. I man, I’m pretty much home bound.
Sue became ill in her early fifties with pulmonary fibrosis. Very quickly life changed for us. We’ve been together since we were teenagers. We were devastated when we received the diagnosis of IPF. I’m always reminding myself that Sue used to be able to do everything and now she can’t I have to be an extension of her. She still has the same strong opinions, emotions and aspirations. I’ve learnt to be more patient, tolerant and understanding.
Caring simply means you just help them achieve whatever it is they want to do. Being a carer for your partner pulls on the heart strings and tests all the bonds you have relied on throughout your life together. I feel empowered by understanding the disease and treatments.
For me and my wife the two most difficult parts of living with scleroderma are 1) the sense of uncertainty and 2) the coordination of care among the many doctors and facilities involved. The sense of uncertainty makes us fearful of the real and imaginary symptoms that we see in me.
‘I got told off by my daughter for cutting the grass. But I got to do it. I got to do it when I get the chance.’
‘My daughter knows, my husband knows. He has been very protective in a loving way. My son, haven’t told him.’
‘My son had a shock when he saw me with the oxygen.’
‘Eventually she is talking about us moving downstairs and staying downstairs once it gets to a certain point—well I don’t want that. I’d rather take 10 minutes to go upstairs to go to bed than staying downstairs all the time.’
‘What consumes my husband controls my life as well.’
‘It soon became about logistics and practicalities and how our home would need to be adapted with a bedroom downstairs and wider door frames and ramps for wheel chair access.’
‘At the time, we didn’t feel the full force of what this meant. It was scary to think that my dad had only possibly 5 years to live, but because he hadn’t changed much (apart from a consistent cough), the terminal diagnosis hadn’t quite hit home ... His use of oxygen tanks was a big adjustment at first, and at times was difficult to see. But we knew he needed it to continue being as active as possible.’
‘Pulmonary fibrosis has hit my family hard, but the magnitude and impact on families grows exponentially when you multiply that by as many as 500,000 deaths every decade in the US alone.’
‘I’ve had some truly frightening experiences where coughing has caused my throat to close and, for a few seconds, I can’t inhale or exhale. This lasts only a few seconds but it is terrifying for my family.’
‘It affects your whole family ... our children, our extended friends, everybody’s lives were tinged by the illness that I was going through.’
‘I have tried to talk about this (end of life care) with my wife, but she interrupted me suddenly; she does not even want to hear me talk.’
In Spain it is necessary to improve early diagnosis of IPF. It is also necessary to create specific Multidisciplinary Centers that would allow an adequate follow-up and treatment to those suffering from the disease.
As there is no national transplant waiting list, it is not guaranteed that the person who needs the transplant most will get an available organ first.
One of his biggest fears is that he will miss the ‘window of opportunity’ to go on the waiting list. If his health declines too much, he might no longer be suitable: he needs to be sick enough to need a transplant, but not too sick to prevent the transplant going ahead.
Most oxygen providers will provide a home concentrator and either tanks or a portable oxygen concentrator (POC). Initially I ended up purchasing a POC out of pocket because it was difficult to get through insurance. Tanks are very heavy and, depending on your flow rate, you may have to carry more than one tank with you. Last fall I was prescribed a POC with a low continuous flow capability and it took over two months, and several follow-up calls by me… oxygen deliveries can be unreliable and sometimes require that you be home for a full day.
I wanna know how I’m doing you know, I want to be able to see this is what it was last time I did the test and this is what it is now.
I’ve got every reading I’ve ever done for the last 3 or 4 years. So, when my boy graphed it on the computer so we can see exactly where I’m at, how I’m going, you know whether my gas transfer is going up or down, do that type of thing. And within say right, maybe I gotta do a bit more exercise or whatever else, it’s a terrific gauge.
I sort of keep my phone, does the counting actually, number of steps and kilometres walked so, that’s a rough guide to how much I’ve done.
Sometimes the caregiver really knows … living with someone, I think you have a better perspective. And I think it’d be valuable for you to get the caregiver’s opinion … because they’re with them all the time.


### Patient experiences and needs related to evidence/translation ###

I hope my participation in the [Pulmonary Fibrosis Foundation] Registry can ultimately help other patients understand and plan for what their disease may bring. How do you learn to live a fairly productive life when you have a progressive disease and there’s no playbook on what to expect? By filling out quality-of-life questionnaires for the Registry, I want to provide enough data points so that healthcare providers can help their patients work through their struggles.
Nevertheless, questions remained, and one of them was worrisome enough to get people to consider an unconventional approach to follow-up research. The problem is that the results that come out of drug studies in mice almost never translate to humans… As a result, we have close to thirty failed drug trials in pulmonary fibrosis alone.
I take part in every research study I can… Maybe that information won’t help me, but it may help someone else, maybe my children or grandchildren.
Now for the pulmonologist, the breathing is that big target because it’s a measurable thing, but for me the improvement, the quality of life is, can I walk upstairs better, can I perform things better, can I take a shower and put on clothes at the same time within the same hour? Those are things that are important to me.
I struggle with the pulmonary function tests just as my illness progresses the pulmonary function tests become more and more challenging … It’d be great if … somebody could ... come up with a better … a different PFT
[In relation to caregiver assessments and caregiver reported outcomes] Sometimes the caregiver really knows … living with someone, I think you have a better perspective. And I think it’d be valuable for you to get the caregiver’s opinion … because they’re with them all the time.
But before you start (pulmonary function tests), just say Jim, Mary, Tom, we have no idea where you’re at. Just give us your best shot. You know what I mean? A little bit of a pep talk probably wouldn’t hurt.

### Patients’ perspectives on design of clinical trials including patients’ experiences on trial site visits and logistics ###

There’s 10 plus visits in a year … then at each visit, it’s at least a two-hour visit … that’s a fair amount of hard work as a patient. And I’m just not sure that I’d be up to that amount of work … I probably would still go forward with it … but having looked at what my requirements are for it, it’s just dampened my enthusiasm of it. I’m still going forward with it, I’m just not as excited.
Getting to the weekly visits, that can be difficult. My wife is in a wheelchair.
I suffer from chronic fatigue … Questionnaires are taken at every visit so that’s going to be a 30-minute function so I’m not sure that, that needs to be done frankly.
I think two hours would be probably the outer limit of length. I could do it, but I just think anything more than that is pushing it ... you’re going to be dealing with people who aren’t always having a good day.
There’s a lot of blood tests ... And when I saw this, I thought, wow, that’s a lot. And each time ... each time you get blood tests right for example.
I was wondering about that and the blood test frequency, a little bit high I would say. when I saw this, I thought, wow, that’s a lot. And each time ... each time you get blood tests right for example.
The total, the 52 weeks didn’t seem like a deal breaker but the greater the number I think the more people would find more onerous.

Chronic fatigue syndrome; I only have five to six hours a day of energy to complete activities … so for me … the day is short for me. And chronic fatigue is common amongst ILD patients, so we got to think about that.
Participants don’t have to change their medications or how often they see their physician.
If you have a big site … you may have to walk a mile … I can’t do a mile … I don’t typically have a wheelchair, but I might need one for that … it’d be nice to get a shuttle from the parking lot, or get dropped off at a door, or something to know where I’m going and how I could best get there.
Either bus passes, or city transit, or parking … or the city passes, or … Uber credits or something along that line is good, because it’s hard for some people to get around.
As long as they’re, I would say, probably morning to early afternoon [discussing visits] is better for most of the patients. His body starts shutting down probably by three in the afternoon. He starts
going downhill.
It would help that in working with the study coordinator, that they would have a little bit of flexibility. And if I would have a vacation planned [at] visit 6 that we could, we can move it out a week, but still be within that time period, and do something. So, we just have to see.
So for instance, I don’t know to change my time or the day of my visit, but as I say, I will do my best. I will do my best to stick to the visits that they will schedule for me.
I’m big on reminder emails … remind them of visits and remind them at this visit here’s what we’re going to do … just to set the expectation that it’s going to take a while. And then … if there’s anything that needs to be done beforehand. If you tell them up front … maybe that’s a little bit helpful.
If we had an agenda. If we had a schedule, I would feel less anguish. I would feel more safe.
I would do whatever I needed to do, but I just thought it was much more reasonable than having to travel far, and the fact that you can do some things on the computer. I liked that.
Yes, I would benefit indeed [from virtual visit option] because not all the days are the same.
I guess the most positive thing is that it can be done from home.
I prefer visits. Well, I prefer … the person I care for to undergo tests for better control, for better monitoring … But if visits were to be virtual, my scoring will not change.
Because everything that has to be done through the computer is more difficult for me, so I’d rather go to a place in person and get the tests done. I come from a different generation.
I would expect them to tell me potential adverse events … long term adverse events. I would like them to let me know about them … I always take into account whether it will be detrimental as well for me in terms of adverse events.
Maybe test results, anything to do with the test results in the trial … if the results could be sent to my PCP and be seen by my PCP as well, that would be great.
Well, I guess that my primary care physician would like to hear about the … regular testing that I will be undergoing throughout the clinical trial … he refers me to other doctors … therefore it’ll be very important for him to know about those tests, the outcomes, the results, how those tests were done.
Well the side effects, diarrhoea. I was wondering about that … that really stuck in my mind in terms of concerns.
[In the context of discussions about side effects] In principle, I’d like to know everything … I’d like to know if there has been secondary effects, I guess this has already been tried on humans. That’s what I’d like to know.
More than the primary care, I think the pulmonologist would be the one who should be informed. The specialist … I think I would be telling him that I’m getting a trial to see what his opinion is. Once I’m in, I would like for him to have the tests and results at his disposal.
It’s a really good to have that person [discussing study care navigator] that you know you can always get a hold of because sometimes the doctor … is not available. So having that contact there that is always available … I think to navigate everything and again takes some of that stress level off of you.
I think it’s all about mental health and I’m thinking about maybe a psychologist, psychological support, having some visits with a professional where she can really explain or share those concerns. I think it goes around the same idea of the peer group or being linked to this trial beyond just a medical fact, the fact that it’s personally and psychologically giving her something.
Well when you have a peer group you can feel more supported. Psychologically it’s really helpful at least for me as a caretaker and for my mom as a patient to be able to share her experience with someone who’s going through the same thing, I think that’s very helpful.
At least in studies that I’ve seen, I’ve never seen study partner caregiver compensation. I highly encourage it because they have costs, and they have skin in the game.
I’m just worrying if something like that happened, and he needs to be hospitalized, and he needed to be on more prednisone than he normally is ... I see that it’s not permitted.
No, the thing that worried me was, if he had to stop any of his meds that he was on now ... when I saw the list of meds, he wouldn’t have to stop any of his regular meds and be taking placebo. I was OK with that even if he was just taking placebo.
We went to different hospitals, and they said it’s a sequela of COVID and so we can’t do anything with that. And they don’t have any experience so we can’t do anything.
They say you are like this and that’s how you are going to end up like. And you think, what’s going to happen in between?
Well, I’m in the dark about what you can do to help the condition.
Lack of information always causes more worries.
If I was diagnosed with lung cancer it would have knocked me over, but knowing I have a fighting chance as things are, it’s not bothering me.
I was pleased it wasn’t cancer.
Doctors should be sensitive when delivering a diagnosis and prognosis to patients. The delivery of a diagnosis should not be rushed as the news is life changing. Time should not be an issue.
If somebody says to you it’s mild, moderate or severe, all that immediately goes through your mind is does that mean I have a short life or a long one? It doesn’t tell you what you’re faced with.
It took me years to get diagnosed and I know this can isolate you so much because you just feel you need to know what is wrong with you so you can deal with it.
It is extremely unsettling to have a disease that people say is rare and incurable. The research that I did on ILD was also hopeless pointing to low 5 and 10 year survival rates ... I had a lot of unanswered questions: How would my appearance change? Would I suffer? What other organs would be impacted after my lungs? What did I need to do to ensure the financial stability of my family? What did I need to do so that my family did not need me anymore?
The diagnosis was communicated to me in a realistic manner. There is no treatment and no cure but – I remember the doctor emphasised that but – but there is a medicine that gives encouraging results.
Being told you have this and there is nothing you can do is frightening. I was told that I wasn’t suitable for transplant, so I felt in a position of no hope. I just had to sit down and take what was coming with no hope, and that was the worst of all.
I was told my life expectancy was two to five years and then I was scheduled for a check-up for six months later. I was referred to a physiotherapist to learn breathing exercises. That was it – no other information or support was given.
More thought should be given to the psychological impact of being diagnosed with this devastating disease.
We were not given a lot of information. Most of what I learned was from attending a support group. I was told when the illness was diagnosed that it was terminal, which came as a shock to both me and my wife.
I didn’t know what to do initially ... I tried looking at the internet and the information that was available managed to confuse me even more.
I think at diagnosis, obviously, you need to be told that this is not curable and it’s progressive and it will end your life.
I think information can be provided in [the] way of a website, or even at the respiratory clinics, little leaflets for people to take. For example, having problems with exercise or keep healthy eating or, you know, when you’re feeling down, these are the resources.
[Information] should be available to you very early on, to put you onto the right track of exercise and diet.
I think that [what] the website should include, is that you’re not going to keel over from just exerting yourself a little bit. I was really frightened to do any exercise in case I made things worse.
The missing link for patients is the lack of information among the GPs. They are not informed about the disease and, in turn, they cannot properly advise patients or refer them to the right specialists.
We knew nothing about pulmonary fibrosis when mum received her diagnosis and immediately researched it. Google was not kind and we couldn’t believe something so cruel and terminal was unheard of. Anyone we spoke to about pulmonary fibrosis also hadn’t heard about pulmonary fibrosis and we soon became educators. This really needs to change. There has to be more awareness and support for pulmonary fibrosis sufferers and their families.
It was so hard to find information and any doctors or nurses we did talk to had never had a patient with IPF before. So what they maybe heard about or they were comparing it to COPD or other diseases that it really can’t be compared to. There was no information and it was really hard to find, and it felt like we were completely on our own. From what I found online, it was scary; but I figured if that was the case, then her doctor would tell us.
[In the context of advanced care planning and end-of-life care] I don’t think the GP knows enough about IPF to be honest because in my experience, they have heard of it but they might have one patient with it but it seems like by the time people are diagnosed, they are too far gone; or it just seems like it’s getting easier to diagnose ... Even the Palliative Care Nurse didn’t know about the oxygen testing and didn’t know about lung disease at all ... So I think they need to be more educated.

### Patient experience and needs related to drug and delivery ###

Keep working with the doctors. I think a lot of the issues in the medical community is that it is still kind of an unknown disease. Especially in those smaller centers, where maybe the doctor has never seen this.
I run everything through my GP first. So, once he found out that I have this, he did not know about it. And he told me, I’ve never had a patient like this before. I’m going to learn all about this. And he spent the time learning. He learned on the journey with me, so that’s pretty cool of my doctor.
I believe it is important that healthcare professionals are honest with patients and families about the risks, so they can base their decisions on the facts.
The support my father got was very good, especially from the specialist nurse who helped him to manage his condition.
I have a good relationship with both the nurse and the respiratory therapist. They’ll decide on a plan, and then the nurse communicates with [the ILD physician] or [the rehabilitation/palliative respiratory care physician]. They handle all of that really well. We [patient and the nurse] had a conversation around this ... about how well our relationship is working. And I trust her [the nurse] implicitly to be communicative with [the ILD physician and the rehabilitation/ palliative respiratory care physician].
You have full involvement and knowledge in what’s going on. It’s very clear to us she’s [the ILD physician] going to see us through to the end of this and it’s her primary concern to do the best for us that she can. And that’s a very good feeling—a feeling of trust and security. We’re just feeling very lucky to have somebody like her and her entire group because they all have that same approach or philosophy to it all.
We were terrified to have those conversations but once we knew for sure that was what was happening, we made it a point to talk about it openly and talk about her fears and concerns and that was huge. That was one thing that Dr. J nailed was being up front. What do you want your death to look like? How do you want to die? What scares you the most about it? And for my mom it was—she didn’t want to be drugged. She wanted to be conscious right until she couldn’t be anymore so they kept that in mind.
Do I just go into hospital and I just gasp until I die? But, being reassured by them that I can have a home death, there can be drugs that can help to calm me down, and people there ... Yes. And how caring and supportive the doctors can be. That there are drugs, medications that they can give you to calm you down, so it’s not scary at the end.
I ended up in hospital for a week on IV antibiotics and a chest tube. Plus they produced a Green Sleeve, which I’d never seen or heard about until then. I can see their point. They want to get it done before you end up needing resuscitation. But honest to god if you’re short of breath, got pains in your side and feeling lousy, it’s not necessarily the time to be asking you questions about levels of care.
And where you want to die and what the different types of experiences look like—so dying in a hospital versus a hospice versus at home. And with each of those different scenarios— the positives and negatives about it ... So it is really just about having those conversations and so people know what to expect and they are not blind-sided by it and making sure that the hospice is willing to work with them. And if they are going to stay at home, what resources are available to them?
I think everything should be addressed … There’s so many facets to a terminal disease that you don’t even think about before you’re diagnosed ... It’s really hard ... By being honest. By laying it on the line and telling me this is what we do. This is what we can do. This is what could happen ... It’s very important. It’s one of my biggest fears. But now that I know a bit more about it, I go to a support group that Dr. J spoke about ... and I do feel much better just knowing. I was afraid of the end. Of how violent it could be. From things that I’ve heard … it was very calm and very reassuring.
My hope is that oxygen companies will realize that oxygen needs are not a one size fits all across patients and will work more closely and compassionately with patients to ensure their oxygen needs can be met in a way that allows as much quality of life and as little frustration as possible.
I know that since my transplant there has been great strides with new drugs to slow down the progression of IPF and research continues to find a cure. I only hope and pray that this will be made possible soon so the need for transplantation will lessen.
There was a general lack of understanding of the condition’s progression by GPs and others, and no services available to support patients and their families at the end of life.
In my view, the health system let my mother, and us as a family, down badly at the end of her life, due to disjointed services, breakdowns in communication, and the fact that there was no one person to coordinate health and care services. It was truly a traumatic experience.
I do feel that unlike cancer patients, pulmonary fibrosis sufferers do not receive the same emotional and mental support to help prepare them through the journey ... Explaining pulmonary fibrosis to anyone was a challenge… This really needs to change as it’s deeply unfair and seems borderline discriminatory. Anyone going through pulmonary fibrosis needs to be supported and understood, just like any other terminal illness. Getting any support for mum was hard because it felt like her condition was not taken seriously until it became dire.
My father died of pulmonary fibrosis when I was in college, and that was more than twenty years ago. In all those years, there’s been no change in a patient’s experience with the disease.
‘It took around 2 weeks for her to be diagnosed after being taken to A&E at Hammersmith Hospital, London when she felt her breathing was not right. I remember being told at the time that her lungs were too weak for a biopsy to be done to confirm her diagnosis. Shortly after, she was given a prognosis of 5 years. This was a massive shock and we felt totally winded. It was unexplainable for us and felt surreal.’
‘I first mentioned my cough to my GP at my annual health check in 2013. The X-ray came back clear. But a year later, another X-ray showed something on my left lung. Lots of tests were inconclusive, but finally the results of a surgical lung biopsy indicated I had IPF.’
‘It took 2 years of back and forth to get to this stage!’
‘It took around 2 weeks for her to be diagnosed after being taken to A&E at Hammersmith Hospital, London when she felt her breathing was not right. I remember being told at the time that her lungs were too weak for a biopsy to be done to confirm her diagnosis. Shortly after, she was given a prognosis of 5 years. This was a massive shock and we felt totally winded. It was unexplainable for us and felt surreal.’
‘I first mentioned my cough to my GP at my annual health check in 2013. The X-ray came back clear. But a year later, another X-ray showed something on my left lung. Lots of tests were inconclusive, but finally the results of a surgical lung biopsy indicated I had IPF.’
‘Then we moved and I saw a new GP. I was sent for a CT scan almost immediately, which showed that there was damage in my airways. I was told I had pulmonary fibrosis and referred to hospital. After a lung biopsy and further tests I was finally diagnosed, three years after my first visit to the GP.’
‘…but had always been diagnosed as something else, such as pneumonia. This unfortunately led to countless, useless prescriptions of antibiotics; to post-nasal drip; to possibly tuberculosis (simple tests proved otherwise). Luckily, my new and astute physician decided that the problem was greater and a referral to a respirologist confirmed IPF.’
‘I was referred to a respirologist and put on a high dose steroid, and a bronchodilator inhaler. Several tests followed and finally in December 2010, based on a lung biopsy, I was diagnosed with IPF that had already reduced my lung function by 40%.’
I wanted to get better; I wanted to get back to normal
The doctor told me that I might be cured. I really want to be cured.
I wanted to live longer and the chemo, Gemzar, was very harsh and I didn’t think I had much chance to live much longer with the harsh chemo that would take me back down again. That was a scary chemo. Basically, I just wanted another chance and hope.
We all said, if this helps people who do actually have it [breast cancer], then we feel, indirectly that we have done them a service.
I would do it for anything; I wouldn’t care if it was paid or not paid.
Rather than feeling fearful of being allocated to that drug or feeling like being experimented on, I thought anyways I’d been told that there was no cure and extending survival was the only way, and thought it would be lucky if allocated to the new drug, or even if not, I would be able to receive the standard of care, while receiving it for free of charge, so I decided to go ahead.
Looking back, now I believe the first contact could be more detailed, even though by phone, just to have a better idea, then later more specific details could be provided.
I looked at the name of the pharmaceutical company. I tend to look at the name of the pharmaceutical company because, at the end of the day, you are putting yourself forward and you want to make sure that it is a reputable company. With the Internet at our fingertips I am not a great Googler, a Google person but, I did look at the company.
It is just a whole bunch of paper. I’ve done trials with the same company and all my information is on their system, but I have to fill out a brand new for every time. I don’t really know why?
But, if you don’t get in, I have had that too – so that is not good, you think oh god, what’s wrong?  And they’re like, no, no, there is nothing wrong, it’s just a particular study and your bloods didn’t fit, or something like that. Because, if there was anything crazy [wrong] I think they would tell you.
Would accept it anyway, not only to help others but also because it’s an opportunity to learn more about the disease. What are the risks of the disease? I would like to learn more about my disease
Perhaps the cost of overall treatment will be free of charge. All the medical cost will be free of charge.
The first thing that would come to mind for me would be some sort of side effects that could make the whole thing worse than it is and I may get some additional disease that I really don’t need.
I don’t know if I have to be locked in the hospital and can’t see my family members. This is my concern. If I am isolated inside, I can’t connect with external world.
It’s not just a financial cost, there’s also a time cost that you need to take into consideration as well. There’s a big difference between what I do now for my treatment, which is basically very low maintenance versus some clinical trials can be very time intensive where you need to be in hospital for a period of time, whether it’s infusions or side effect monitoring or that sort of thing.
Right now I probably wouldn’t join a clinical trial because I am trying to have children next year
There will be a need for someone who is capable of giving a thorough explanation to clarify for the patients about the significance of the clinical trial, impact on the body and what follows after the trial.
I think there are a couple of things that could happen. I think one is being very transparent about what is covered and what is not upfront. So I think that’s number one. Number two, I think at some point we’ve got to look at how do we do remote, I know part of a trial has to be controlled, but how do you establish that level of control but also establish remote site capabilities where people aren’t travelling the same distance to get to treatment and to the clinical trial.
It is just a whole bunch of paper. I’ve done trials with the same company and all my information is on their system, but I have to fill out a brand new for every time. I don’t really know why?
But, if you don’t get in, I have had that too – so that is not good, you think oh god, what’s wrong?  And they’re like, no, no, there is nothing wrong, it’s just a particular study and your bloods didn’t fit, or something like that. Because, if there was anything crazy [wrong] I think they would tell you.
The doctor was very competent, and it was not a 10-minute appointment, it lasted much more. So, the feeling I had was that the doctor really wanted me to improve. He wanted this to work out.
After participating in the clinical trial, I began to think positively. It’s like catching the lifebuoy in the vast ocean. I was back to the positive and optimistic mindset like I had before. I didn’t feel sad at home or like locking myself in my room. I regularly visited hospital to talk with doctor and nurses. When my disease got stable control, I felt happier.
They are so attentive and caring. Sometimes you arrive there feeling a bit low and they pay special attention to you.
Everyone was so polite and they really want to help and the treatment you get at the hospital is ace, it is really good.
You know it’s a clinical research laboratory but the people are very good, they are very kind. You are not treated like somebody on a conveyor belt.
I was a little nervous at the time of the first dose, I had an accelerated heartrate and the study nurse noticed I was anxious, she calmed me down and made me feel important.
At the time I was just worried about the side-effects and the adverse reactions and that the impact on my life would be very troublesome…the physician told me not to worry too much and trust myself and the manufacturer…I didn’t know if I could completely trust the Dr or the medicine, I could only know after trying it.
There is the rigorous schedule and the side0effects, and just waiting to see if there are any side-effects.
I was worried at the beginning, worried that the medicine would impact my body and make me feel uncomfortable, but it wasn’t such an issue after a week once I saw that I could tolerate and overcome it.
After taking the medicines for 1-2 weeks my thinking became more relaxed, I felt better without discomfort or impact.
‘What does the long-term testing and visit schedule look like?’
‘Will I be able to continue the treatment after the study?’
‘Are there any preliminary results from the trial?’
My blood pressure was measured each time and I’d had to have it measured until it was within the approved range. I found it cumbersome. And I had to speak to the coordinator before I could receive the drug and I found it time-consuming and wasteful.
It’s the waiting time. It takes me a whole day. I get there in the morning, through the lunchtime and until 2 or 3 o’clock. The waiting time was extremely long when I was receiving the intravenous infusion and for the blood test results. …Once I am told that I can go ahead to receive the drug, they will start thawing it and it takes 30 min, but it’s longer than that, it takes one hour.
I had to go back every 2 weeks and again, I was more tired the first 2 days afterwards and things like that, this is what was getting online with other people, we were having the same thing; we were really happy we were living but we wanted to live our life. To be stuck into this schedule was getting tedious.
But before you start [pulmonary function tests], just say Jim, Mary, Tom, we have no idea where you're at. Just give us your best shot. You know what I mean? A little bit of a pep talk probably wouldn't hurt.
I struggle with the pulmonary function tests just as my illness progresses the pulmonary function tests become more and more challenging … It'd be great if … somebody could ... come up with a better … a different PFT.
Now for the pulmonologist, the breathing is that big target because it's a measurable thing, but for me the improvement, the quality of life is, can I walk upstairs better, can I perform things better, can I take a shower and put on clothes at the same time within the same hour? Those are things that are important to me.
Sometimes the caregiver really knows … living with someone, I think you have a better perspective. And I think it'd be valuable for you to get the caregiver's opinion … because they're with them all the time.
I prefer to experience diarrhoea than a headache as my line of work requires lots of concentration.
If they are all severe, diarrhoea is the hardest … every moment of the day going to the toilet can be embarrassing and hits that physical and pride level.
The most common thing that a lot of people can’t manage is the diarrhoea, because you’re literally at home and you can’t go anywhere.
I only have five to six hours a day of energy to complete activities … so for me … the day is short for me.
I think two hours would be probably the outer limit of length … I just think anything more than that is pushing it ... you're going to be dealing with people who aren't always having a good day.
If you have a big site … you may have to walk a mile … I can't do a mile … it'd be nice to get a shuttle from the parking lot or get dropped off.
think it goes around the same idea of the peer group or being linked to this trial beyond just a medical fact, the fact that it's personally and psychologically giving her something.
I've never seen study partner caregiver compensation. I highly encourage it because they have costs, and they have skin in the game.
I completely trusted my doctor to make the right decision for me.
I had researched online about anti-fibrotics and asked my doctor to prescribe.
I looked it up on the internet, checked it out. The description is a little bit more dire than what my experience is but you know I looked it up on the internet to read up on it.
The doctor said I was free to search for information on my own, however he asked me to share the information with him.
I didn’t know what I had, that it was bronchiectasis. [I felt] desperate…angry…sometimes depressed, because I simply don’t know what’s behind it or if you can treat it or not. Phases where I was angry, dissatisfied, depressed. [I’d] wonder why no one cares and no exams are being initiated. Lots of fear.
I was just depressed. I was frustrated. Honestly, being sick all the time was depressing.
I was relieved to have a partial answer to what was going on with me, but also kind of concerned like ‘what does this mean for me, and how’s it going to affect my life?
[The] biggest challenge is finding the right doctor to really find out what was going on. One doctor told me there was nothing wrong with me and that I was lazy. Made me frustrated and angry that they wouldn’t listen to me.
I was angry that I wasted time with the cough and cold medicines. Maybe I lost lung function?
If I had had a referral to a university hospital, I may have had better treatment in my 20s.
I just tell people that I have asthma. It is much easier that way.
When speaking to my friends, I use the term ‘chronic bronchitis.’ The average person will probably not understand ‘bronchiectasis.’
Going online… WebMD or Medicine. This information is helpful. Also looking into research being done, reading articles about advances being made. I found that to be really helpful.
Bronchiectasis is in my face every morning, every night, all the time, because I'm coughing and I'm having to explain my cough […] I'm coughing up stuff, everybody’s thinking ‘covid covid’. With that in mind, it makes it that much harder to not only have this condition, but a condition that allows people to sort of discriminate against me. Something I haven’t experienced until now.
I am particularly nervous when changing seasons, I am even more prone to cough, and if I cough, I am prone to have sputum. And at work, if you frequently cough in the office, people are going to look at you with weird looks.
The cough is very painful, I often feel one or a few very strong local pains in the lungs, like someone stabbing me with a knife, and my whole body is sore and tense.
Boehringer-Ingelheim Version 1 - March 2023
I’m fighting breath each time I breathe. I know that each exacerbation further damages my bronchi and lungs.
I feel pretty neutral about managing daily life. It’s a lot of work, but I manage to have a pretty good quality of life. There’s a balance of negative and positive.
I take it one day at a time and don’t take on long term projects. I just prefer to have the day as it comes. I enjoy moments, but I have a hard time projecting into the future.
At the moment of exacerbation, my life is put on pause, and to a certain extent the life of my family too. I’m in pain, I choke, I feel alone, sad and sometimes scared.
The toughest part for me is knowing it’s going to get worse no matter what I do. It’s like being in a tug-of-war with a stronger person. You’re eventually going to end up losing. It’s just a question of how soon.
I try to clear stuff out by doing breathing treatments that help me open my lungs and loosen secretions, and then I do physical therapy which tries to help me cough up mucus, but inevitably it's not going to get all of it and what's left causes infections.
I can do nothing other than avoid crowded places and protect myself against COVID.
The Facebook group is a big source of support. When I want someone else’s opinion who’s also living with it, I definitely turn to them. Maintains hope and it’s a social connection. Knowing I’m not alone is incredibly valuable.
I realize that each and every day, I ingest a lot of different products. However, without them, I would not feel as well as I have been feeling up to now. So, my feeling is stuck somewhere between ‘sadness’ and ‘I have no choice.’ Others are not as lucky as I am to have a treatment that can keep them alive.
It's just overall, it's just frustrating for me to have to take so many medications. I know the treatment is supposed to help me, and I was glad to be doing something to hopefully help me get better, but then also, ok I’m going to have to do these inhalers the rest of my life.
And I think that the pre-read is really much easier to understand for patients, but that can be expanded with more information on the drug, because that was actually what I was missing in the pre-read, how is the drug working? How is the effect of the treatment?
I was a little confused on the pre-reads about the adverse events and so forth. So will you speak anything more about that now, or just clarify what adverse events you’ve seen so far?
I think the schedule is very long. Even for healthy patients who are in a good condition. I mean, from 08:30 to 16:30, it’s a whole day.
If it’s an older patient, it can be a problem with the bigger ones, because sometimes people, they have problems swallowing. The 10 milligrams and even the 5 milligrams can be difficult to swallow for a patient. And even younger patients, sometimes people have problems swallowing these big pills.
I think you need to really clearly label A, B, and C. And say you’d have to take one from each. You don’t take three from A on one day, and then three from B the next day.
I think if you’re going to encourage people to collect every tiny little bit of sputum over [a] 24-hour period, then I think the non-transparent one will be more socially acceptable if you’re on public transport or something like that.
I think a guide on patient groups or even rare disease groups in each geography would be helpful.
I have seen a lot of science and a little bit of lay. So, because you are speaking for a huge population of patients, most of us don’t have any scientific background.
Some words should be explained [more]. You should not avoid the terminology, but more explain it to the patients.
I would suggest not to give too much information. They already receive the patient information leaflet, which should be [as] clear as possible and as understandable as possible.
I would suggest not to give too much information. They already receive the patient information leaflet, which should be [as] clear as possible and as understandable as possible.
In my opinion, [information] during the clinical trial could be important. Because if I were a patient in clinical trial, I would like to know which are the adverse events and in case of one of these what to do and where to go … This is important for people in clinical trials, I think.
One of them is three a day, I mean three twice a day, and, and the other two are like one a day. One is like, a blood pressure medicine that deals with [the] inflammation. It helps with the inflammation around my heart and things like that. But the others are just, you know, it’s just a dosage. I would prefer a pill, because I’ve seen people go through infusions and things like that. That can be tiring, that can be stressful.
I wish people knew how little is known about scleroderma and how debilitating this disease can be for someone suffering from it. In general I wish more research could be directed at rare diseases such as scleroderma.
Partners need to be involved, sometimes they have more problems than the patient. Important aspect as a caregiver, my husband died last year of lung fibrosis, the greatest problem was the end of life care, end of life is a very difficult experience, the patient faces a very difficult phase of their life, family was affected, the best situation was him being at home, but the phase end of life, the assistance should have a great attention than it has before now. A single approach from EU states if possible. Family or spouse needs to be included.
I wish people knew how little is known about scleroderma and how debilitating this disease can be for someone suffering from it. In general I wish more research could be directed at rare diseases such as scleroderma.
Partners need to be involved, sometimes they have more problems than the patient. Important aspect as a caregiver, my husband died last year of lung fibrosis, the greatest problem was the end of life care, end of life is a very difficult experience, the patient faces a very difficult phase of their life, family was affected, the best situation was him being at home, but the phase end of life, the assistance should have a great attention than it has before now. A single approach from EU states if possible. Family or spouse needs to be included.
Looking back to my experience of ARDS and being ventilated the medical staff were all brilliant beyond description. The professionals able to answer questions eg will I be able to walk again, breath more freely, be independent etc were the physios. They explained how being in a coma and not moving causes every single muscle in the body to lose strength.
It has been a bit of trial and error in getting it right. I tried after breakfast when taking my other medication. Not good. Then I tried taking HCQ after breakfast without my steroids etc. Remember I can’t take my gastric reflux medication (Lansoprazole) until at least two hours after taking HCQ, so have to space that as well.
I don’t feel as held down by the lack of breath issue and stuff. Not that it’s a perfect solution but it’s just – one thing that I always say, even if I feel 10% better, to me that’s like 100% better, because I’ll take it, no matter how minuscule it is [… ] So far, so good, because of the benefits. I’m kinda outweighing the negative. That’s basically the high of my current treatment [unspecified].
I actually broke out with shingles. And that could have possibly been one of the side effects, because the medicines [unspecified] weakened my immune system. That’s when I just started out. That’s something that concerns me with the medication, is that it did weaken the immune system, and it seemed to me that whenever I would take that particular injection, that something, something happened. That would break out with one thing or another.
After I have those infusions [unspecified], I have a lot of abdominal pain, I have a lot of nausea, vomiting, and diarrhoea. Those are kind of like the mainstay for maybe the first month after I get them. And then little by little, it subsides more.
I started chemotherapy for six months ... My hair and beard thinned and I lost about 30 pounds but my appearance did not change noticeably. I was fatigued, could not eat, felt dizzy and disoriented, my voice became softer and I could not concentrate but soon I learnt the chemotherapy cycle and adjusted to it.
I have scleroderma with pulmonary fibrosis and the usual low motility, reflux issues… I am now on Mycophenolate along with prednisolone, folic acid, B12 and esomeprazole amongst other things. Most things are achievable albeit at a different pace to everyone else, I just had to listen to my body and change my way of thinking.
I am now on medication that suppresses my immune system. I have a blood test every month and a pulmonary function test every quarter. All indications are that my ILD is stable and has not worsened ... My biggest recourse to cope with my disease is my journal. I confide my fears and my anxieties to my journal. In my journal I try to make sense of specific emotions and try to sort through coping mechanisms.
I get very short of breath, the edema is a battle and the water pills just aren't doing their job. I have more reflux and am on Prevacid to help control it. I am also on lots of other drugs; Warfarin, a thyroid medication, Cozaar for the hypertension, and Prednisone. I can't tolerate beta blockers for my heart problems, but I carry nitroglycerin [sic] with me always.
have been on prednisolone for a year now and I think it has helped. I have just finished a 6-month cycle of chemotherapy (cyclophosphamide) which has slowed my deterioration right down.
For nine years I continued to have regular three month follow-up appointments with him and also regular pulmonary tests, CT Scans, blood and urine tests. I tried enough medications to choke a horse but, nothing other than prednisone seemed to help.
My hospital has a scleroderma specialist, thankfully! I had been treated for pulmonary fibrosis there. Some other issues landed me with rheumatology, but they seem to have a few multidisciplinary specialists that see scleroderma patients, which is great.
Am going to a connective tissue consultant… as my rheumatologist consultant thinks this would help me better.
A low dose of Baracitinib for my RA. Apparently some rheumatologists don't like prescribing baricitinib to people with lung conditions but I've been taking it for approx 2 years without any detrimental effect on my lungs so far. My respiratory consultant advised me to stop taking methotrexate which I was on when diagnosed. Rheumy have stopped me taking all other DMARDs too, which I think is because of my pulmonary fibrosis.
My pulmonary fibrosis is due to Rheumatoid Arthritis, the RA is being actively treated but constantly getting pneumonia requiring hospital admissions.
I got the Fibrosis, which is life shortening, by repeatedly being prescribed an antibiotic for urinary tract infection. The anti-biotic is Nitrofurantoin.
I too have pulmonary fibrosis, diagnosed 4 years ago and caused by the drug I was taking for my atrial fibrillation, which was then stopped, but the damage was done. I also have other health problems with my heart and lungs.
I have pulmonary fibrosis caused by the treatment I had for AF. I had amiodarone, which is known to cause lung problems, and then dronedarone, which is safer. Unfortunately I developed pulmonary fibrosis.
Where now, I don't have that sensitivity in the fingertips. It will turn purple, it will hurt but not as bad as in the beginning when I was diagnosed. I used to get sores-I used to get digital ulcers and scars and sores, and now I don't. Fingers turn blue but I don't get the sores.
I was diagnosed with sarcoidosis at the age of 31. At the time of diagnosis I had no symptoms, I was athletic and completely healthy. I had no idea the impact it would come to have on my life. As the years went on I began to have flare-ups. These flare-ups forced me to make many physical and psychosocial adjustments.
Have been diagnosed this by my respiratory consultant due to decrease in my lung function tests.
I was diagnosed with pulmonary fibrosis approx 5 years ago. My respiratory consultant believes it RA related and I've no reason to doubt him as I'm as sure as I can be that my lungs were damaged during a prolonged RA flare.
I have a good relationship with both the nurse and the respiratory therapist. ... They’ll decide on a plan, and then the nurse communicates with [the ILD physician] or [the rehabilitation/palliative respiratory care physician]. They handle all of that really well ... We [patient and the nurse] had a conversation around this ... about how well our relationship is working ... And I trust her [the nurse] implicitly to be communicative with [the ILD physician and the rehabilitation/palliative respiratory care physician].
Because of this I’m trying to fill my time and concentrate on something else worthwhile, the rugby club and the APF charity fit the bill!
We’ve also become involved with the APF support group which is great for connecting with and helping others.'''

final_summary = recursive_summarize(text)
print("\n%%%%%%%%%%%%%%%%%%%%%\n")
print("Final summary:", final_summary)

######### Recursion level:  1 

######### 
Number of pieces:  113
****************************************************
Piece: 1  out of  113 pieces
### Living with IPF ###

I was diagnosed with Idiopathic Pulmonary Fibrosis, also known as IPF. I was only 28 years old.
IPF has been a surprise and a life-altering challenge. I’m nearly 71 years old, and was formally diagnosed three years ago.
At the age of 4, I was diagnosed with juvenile rheumatoid arthritis… we counted ourselves lucky that it appeared I had the presentation where it attacks primarily the joints. We didn’t realize this would later take our journey down another path… My childhood had its aches and pains. After college was when things really started to change. I was diagnosed with double pneumonia shortly after graduating… Blood tests, CT scans and a lung biopsy eventually confirmed a diagnosis of pulmonary fibrosis in 2011. My family and I later found out that rheumatoid arthritis caused my pulmonary fibrosis.

### Sympto